In [110]:
library(tidyr)
library(tidyverse)
library(dplyr)
library(ggplot2)
library(readr)
library(pscl)
library(caret)
library(MASS)

In [95]:
all_models_alight_cv <- function(DT_train, DT_test){
        
        set.seed(123)
        rand_ind = groupKFold(c(1:dim(DT_train)[1]), k = 10)
        
        sm_Poiss <- list()
        sm_NB <- list()
        sm_ZIP_Poiss <- list()
        sm_ZIP_NB <- list()
        sm_hurdle <- list()
        
        RMSE_Poiss = 0
        RMSE_NB = 0
        RMSE_ZIP_Poiss = 0
        RMSE_ZIP_NB = 0
        RMSE_hurdle = 0
        
        k_fold_pred_Poiss <- list()
        k_fold_pred_NB <- list()
        k_fold_pred_ZIP_Poiss <- list()
        k_fold_pred_ZIP_NB <- list()
        k_fold_pred_hurdle <- list()
        
        predictions_Poiss = 0
        predictions_NB = 0
        predictions_ZIP_Poiss = 0
        predictions_ZIP_NB = 0
        predictions_hurdle = 0
        
        ind_Poiss = 0
        ind_NB = 0
        ind_ZIP_Poiss = 0
        ind_ZIP_NB = 0
        ind_hurdle = 0
                       
        for(i in 1:10){
                                           
            df = DT_train[rand_ind[[i]],]
            df_test = DT_train[-rand_ind[[i]],]
                       
            Poiss.fit = glm(alight_count ~ ., data = df, family = 'poisson')
                          
            NB.fit = glm.nb(alight_count ~ ., data = df)
                
            ZIP.Poiss = zeroinfl(alight_count ~ . | hour + service_kind,
                               data = df, dist = 'poisson', link = 'logit')
            
            ZIP.NB = zeroinfl(alight_count ~ . |  hour + service_kind,
                               data = df, dist = 'negbin', link = 'logit')
                
            hurdle.fit = hurdle(alight_count ~ . |  hour + service_kind,
                                data = df, dist = 'poisson', zero.dist = 'binomial', link = 'logit')
                
                
            sm_Poiss[[i]] = Poiss.fit
            sm_NB[[i]] = NB.fit
            sm_ZIP_Poiss[[i]] = ZIP.Poiss
            sm_ZIP_NB[[i]] = ZIP.NB
            sm_hurdle[[i]] = hurdle.fit
                
            k_fold_pred_Poiss[[i]] = predict(Poiss.fit, df_test, type = "response")
            k_fold_pred_NB[[i]] = predict(NB.fit, df_test, type = "response")
            k_fold_pred_ZIP_Poiss[[i]] = predict(ZIP.Poiss, df_test, type = "response")
            k_fold_pred_ZIP_NB[[i]] = predict(ZIP.NB, df_test, type = "response")
            k_fold_pred_hurdle[[i]] = predict(hurdle.fit, df_test, type = "response")
                
            RMSE_Poiss[i] = sqrt(mean((df_test$alight_count - k_fold_pred_Poiss[[i]])^{2}))
            RMSE_NB[i] = sqrt(mean((df_test$alight_count - k_fold_pred_NB[[i]])^{2}))
            RMSE_ZIP_Poiss[i] = sqrt(mean((df_test$alight_count - k_fold_pred_ZIP_Poiss[[i]])^{2}))
            RMSE_ZIP_NB[i] = sqrt(mean((df_test$alight_count - k_fold_pred_ZIP_NB[[i]])^{2}))
            RMSE_hurdle[i] = sqrt(mean((df_test$alight_count - k_fold_pred_hurdle[[i]])^{2}))
        }
                                           
        ind_Poiss = which(RMSE_Poiss == min(RMSE_Poiss), arr.ind = T)
        ind_NB = which(RMSE_NB == min(RMSE_NB), arr.ind = T)
        ind_ZIP_Poiss = which(RMSE_ZIP_Poiss == min(RMSE_ZIP_Poiss), arr.ind = T)
        ind_ZIP_NB = which(RMSE_ZIP_NB == min(RMSE_ZIP_NB), arr.ind = T)
        ind_hurdle = which(RMSE_hurdle == min(RMSE_hurdle), arr.ind = T)
            
        train_RMSE_Poiss = RMSE_Poiss[ind_Poiss]
        train_RMSE_NB = RMSE_NB[ind_NB]
        train_RMSE_ZIP_Poiss = RMSE_ZIP_Poiss[ind_ZIP_Poiss]
        train_RMSE_ZIP_NB = RMSE_ZIP_NB[ind_ZIP_NB]
        train_RMSE_hurdle = RMSE_hurdle[ind_hurdle]
            
        model_Poiss = sm_Poiss[[ind_Poiss]]
        model_NB = sm_NB[[ind_NB]]
        model_ZIP_Poiss = sm_ZIP_Poiss[[ind_ZIP_Poiss]]
        model_ZIP_NB = sm_ZIP_NB[[ind_ZIP_NB]]
        model_hurdle = sm_hurdle[[ind_hurdle]]
        
        model_Poiss_AIC = AIC(model_Poiss)
        model_NB_AIC = AIC(model_NB)
        model_ZIP_Poiss_AIC = AIC(model_ZIP_Poiss)
        model_ZIP_NB_AIC = AIC(model_ZIP_NB)
        model_hurdle_AIC = AIC(model_hurdle)
        
        AICs = c(model_Poiss_AIC, model_NB_AIC,
                 model_ZIP_Poiss_AIC, model_ZIP_NB_AIC, model_hurdle_AIC)
    
        AIC_ind = which(AICs == min(AICs), arr.ind = T)
        
        Model_Name = c('Poisson', 'Negative_Binomial',
                       'ZIP_Poisson', 'ZIP_Negative_Binomial', 'Hurdle_Poisson_and_Binomial')
        
        Models_AIC = data.frame(cbind(Model_Name, c(model_Poiss_AIC, model_NB_AIC,
                                                    model_ZIP_Poiss_AIC, model_ZIP_NB_AIC, model_hurdle_AIC)))
        
        Selected_Model = Model_Name[AIC_ind]
                          
        test <- DT_test[ , -c(4)]
            
        predictions_Poiss = predict(model_Poiss, test, type = "response")
        predictions_NB = predict(model_NB, test, type = "response")
        predictions_ZIP_Poiss = predict(model_ZIP_Poiss, test, type = "response")
        predictions_ZIP_NB = predict(model_ZIP_NB, test, type = "response")
        predictions_hurdle = predict(model_hurdle, test, type = "response")
            
        test_RMSE_Poiss = sqrt(mean((DT_test$board_count - predictions_Poiss)^{2}))
        test_RMSE_NB = sqrt(mean((DT_test$board_count - predictions_NB)^{2}))
        test_RMSE_ZIP_Poiss = sqrt(mean((DT_test$board_count - predictions_ZIP_Poiss)^{2}))
        test_RMSE_ZIP_NB = sqrt(mean((DT_test$board_count - predictions_ZIP_NB)^{2}))
        test_RMSE_hurdle = sqrt(mean((DT_test$board_count - predictions_hurdle)^{2}))
            
        Train_RMSE = data.frame(cbind('Poiss' = train_RMSE_Poiss, 'NB' = train_RMSE_NB,
                                      'ZIP_Poiss' = train_RMSE_ZIP_Poiss, 'ZIP_NB' = train_RMSE_ZIP_NB, 'hurdle' = train_RMSE_hurdle))
    
        Test_RMSE = data.frame(cbind('Poiss' = test_RMSE_Poiss, 'NB' = test_RMSE_NB,
                                     'ZIP_Poiss' = test_RMSE_ZIP_Poiss, 'ZIP_NB' = test_RMSE_ZIP_NB, 'hurdle' = test_RMSE_hurdle))
                    
        Models = list('Poiss' = model_Poiss, 'NB' = model_NB,
                      'ZIP_Poiss' = model_ZIP_Poiss, 'ZIP_NB' = model_ZIP_NB, 'hurdle' = model_hurdle)
            
        Predictions = data.frame(cbind('Poiss' = predictions_Poiss, 'NB' = predictions_NB,
                           'ZIP_Poiss' = predictions_ZIP_Poiss,  'ZIP_NB' = predictions_ZIP_NB, 'hurdle' = predictions_hurdle,
                           'count' = DT_test$alight_count)) 
                      
        performance <- list("train_RMSEs" = Train_RMSE,
                            "test_RMSEs" = Test_RMSE,
                            "Models_AIC" = Models_AIC,
                            "Selected_Model" = Selected_Model,
                            "Trained_Models" = Models,
                            "Predicted" = Predictions)
        return(performance)    
}

In [96]:
get_Model_Coeff <- function(dtfrm){
    Best_Model = dtfrm[['Selected_Model']]
    
    if(Best_Model == 'Poisson'){
        Model_Coeff = data.frame(dtfrm[['Trained_Models']][['Poiss']][['coefficients']])
        colnames(Model_Coeff) = c('Coefficient_Value')
        Model_Coeff$Variable = rownames(Model_Coeff)
        
        return(Model_Coeff)
    }
    else if(Best_Model == 'Negative_Binomial'){
        Model_Coeff = data.frame(dtfrm[['Trained_Models']][['NB']][['coefficients']])
        colnames(Model_Coeff) = c('Coefficient_Values')
        Model_Coeff$Variable = rownames(Model_Coeff)
        
        return(Model_Coeff)
    }
    else if(Best_Model == 'ZIP_Poisson'){
        Model_Coeff_Count = data.frame(dtfrm[['Trained_Models']][['ZIP_Poiss']][['coefficients']][['count']])
        colnames(Model_Coeff_Count) = c('Coefficient_Values')
        Model_Coeff_Count$Variable = rownames(Model_Coeff_Count)
        
        Model_Coeff_Zero = data.frame(dtfrm[['Trained_Models']][['ZIP_Poiss']][['coefficients']][['zero']])
        colnames(Model_Coeff_Zero) = c('Coefficient_Values')
        Model_Coeff_Zero$Variable = rownames(Model_Coeff_Zero)
        
        Model_Coeff = merge(Model_Coeff_Count, Model_Coeff_Zero, by = 'Variable', all.x = T)
        colnames(Model_Coeff) = c('Variable', 'Coefficient_Value_Count', 'Coefficient_Value_Zero')
        
        return(Model_Coeff)
    }
    else if(Best_Model == 'ZIP_Negative_Binomial'){
        Model_Coeff_Count = data.frame(dtfrm[['Trained_Models']][['ZIP_NB']][['coefficients']][['count']])
        colnames(Model_Coeff_Count) = c('Coefficient_Values')
        Model_Coeff_Count$Variable = rownames(Model_Coeff_Count)
        
        Model_Coeff_Zero = data.frame(dtfrm[['Trained_Models']][['ZIP_NB']][['coefficients']][['zero']])
        colnames(Model_Coeff_Zero) = c('Coefficient_Values')
        Model_Coeff_Zero$Variable = rownames(Model_Coeff_Zero)
        
        Model_Coeff = merge(Model_Coeff_Count, Model_Coeff_Zero, by = 'Variable', all.x = T)
        colnames(Model_Coeff) = c('Variable', 'Coefficient_Value_Count', 'Coefficient_Value_Zero')
        
        return(Model_Coeff)
    }
    else if(Best_Model == 'Hurdle_Poisson_and_Binomial'){
        Model_Coeff_Count = data.frame(dtfrm[['Trained_Models']][['hurdle']][['coefficients']][['count']])
        colnames(Model_Coeff_Count) = c('Coefficient_Values')
        Model_Coeff_Count$Variable = rownames(Model_Coeff_Count)
        
        Model_Coeff_Zero = data.frame(dtfrm[['Trained_Models']][['hurdle']][['coefficients']][['zero']])
        colnames(Model_Coeff_Zero) = c('Coefficient_Values')
        Model_Coeff_Zero$Variable = rownames(Model_Coeff_Zero)
        
        Model_Coeff = merge(Model_Coeff_Count, Model_Coeff_Zero, by = 'Variable', all.x = T)
        colnames(Model_Coeff) = c('Variable', 'Coefficient_Value_Count', 'Coefficient_Value_Zero')
        
        return(Model_Coeff)
    }
}

In [98]:
data_split <- function(DT, prop, set){
    n_rows = dim(DT)[1]
    train_set_size = floor(n_rows*prop)
    
    set.seed(123)
    
    train = sample((1:n_rows), train_set_size, replace = F)
    
    if(set == 'train'){
        train_set = DT[train, ]
        return(train_set)
    }
    else{
        test_set = DT[-train,]
        return(test_set)
    }
}

# Route1 Direction 1

## Pre-lockdown

In [99]:
Pre_lock_route1_dir1_bus_stop_124_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Pre_lock_rout1_dir1_bus_stop_124_alight.csv')
Pre_lock_route1_dir1_bus_stop_124_alight = Pre_lock_route1_dir1_bus_stop_124_alight[ , -c(1)]
Pre_lock_route1_dir1_bus_stop_124_alight$board_count = round(Pre_lock_route1_dir1_bus_stop_124_alight$alight_count)
Pre_lock_route1_dir1_bus_stop_124_alight$month = factor(Pre_lock_route1_dir1_bus_stop_124_alight$month)
Pre_lock_route1_dir1_bus_stop_124_alight$service_kind = factor(Pre_lock_route1_dir1_bus_stop_124_alight$service_kind)
Pre_lock_route1_dir1_bus_stop_124_alight$hour = factor(Pre_lock_route1_dir1_bus_stop_124_alight$hour)

Pre_lock_route1_dir1_bus_stop_136_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Pre_lock_rout1_dir1_bus_stop_136_alight.csv')
Pre_lock_route1_dir1_bus_stop_136_alight = Pre_lock_route1_dir1_bus_stop_136_alight[ , -c(1)]
Pre_lock_route1_dir1_bus_stop_136_alight$board_count = round(Pre_lock_route1_dir1_bus_stop_136_alight$alight_count)
Pre_lock_route1_dir1_bus_stop_136_alight$month = factor(Pre_lock_route1_dir1_bus_stop_136_alight$month)
Pre_lock_route1_dir1_bus_stop_136_alight$service_kind = factor(Pre_lock_route1_dir1_bus_stop_136_alight$service_kind)
Pre_lock_route1_dir1_bus_stop_136_alight$hour = factor(Pre_lock_route1_dir1_bus_stop_136_alight$hour)

Pre_lock_route1_dir1_bus_stop_1358_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Pre_lock_rout1_dir1_bus_stop_1358_alight.csv')
Pre_lock_route1_dir1_bus_stop_1358_alight = Pre_lock_route1_dir1_bus_stop_1358_alight[ , -c(1)]
Pre_lock_route1_dir1_bus_stop_1358_alight$board_count = round(Pre_lock_route1_dir1_bus_stop_1358_alight$alight_count)
Pre_lock_route1_dir1_bus_stop_1358_alight$month = factor(Pre_lock_route1_dir1_bus_stop_1358_alight$month)
Pre_lock_route1_dir1_bus_stop_1358_alight$service_kind = factor(Pre_lock_route1_dir1_bus_stop_1358_alight$service_kind)
Pre_lock_route1_dir1_bus_stop_1358_alight$hour = factor(Pre_lock_route1_dir1_bus_stop_1358_alight$hour)

Pre_lock_route1_dir1_bus_stop_1487_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Pre_lock_rout1_dir1_bus_stop_1487_alight.csv')
Pre_lock_route1_dir1_bus_stop_1487_alight = Pre_lock_route1_dir1_bus_stop_1487_alight[ , -c(1)]
Pre_lock_route1_dir1_bus_stop_1487_alight$board_count = round(Pre_lock_route1_dir1_bus_stop_1487_alight$alight_count)
Pre_lock_route1_dir1_bus_stop_1487_alight$month = factor(Pre_lock_route1_dir1_bus_stop_1487_alight$month)
Pre_lock_route1_dir1_bus_stop_1487_alight$service_kind = factor(Pre_lock_route1_dir1_bus_stop_1487_alight$service_kind)
Pre_lock_route1_dir1_bus_stop_1487_alight$hour = factor(Pre_lock_route1_dir1_bus_stop_1487_alight$hour)

Pre_lock_route1_dir1_bus_stop_1582_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Pre_lock_rout1_dir1_bus_stop_1582_alight.csv')
Pre_lock_route1_dir1_bus_stop_1582_alight = Pre_lock_route1_dir1_bus_stop_1582_alight[ , -c(1)]
Pre_lock_route1_dir1_bus_stop_1582_alight$board_count = round(Pre_lock_route1_dir1_bus_stop_1582_alight$alight_count)
Pre_lock_route1_dir1_bus_stop_1582_alight$month = factor(Pre_lock_route1_dir1_bus_stop_1582_alight$month)
Pre_lock_route1_dir1_bus_stop_1582_alight$service_kind = factor(Pre_lock_route1_dir1_bus_stop_1582_alight$service_kind)
Pre_lock_route1_dir1_bus_stop_1582_alight$hour = factor(Pre_lock_route1_dir1_bus_stop_1582_alight$hour)

head(Pre_lock_route1_dir1_bus_stop_136_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,0,0,0.2976687,0.09266714,0.00000000,0
1,weekday,0,0,0.2976687,0.09266714,0.02083333,0
1,weekday,0,0,0.2976687,0.09266714,0.04166667,0
1,weekday,0,0,0.2976687,0.09266714,0.06250000,0
1,weekday,0,0,0.2976687,0.09266714,0.08333333,0
1,weekday,0,0,0.2976687,0.09266714,0.10416667,0


## Data Partition into training and testing sets

* Training set: 80%
* Testing set: 20%

In [17]:
Pre_lock_route1_dir1_bus_stop_124_tr_alight = data_split(Pre_lock_route1_dir1_bus_stop_124_alight, 0.8,  'train')
Pre_lock_route1_dir1_bus_stop_124_test_alight = data_split(Pre_lock_route1_dir1_bus_stop_124_alight, 0.8,  'test')

Pre_lock_route1_dir1_bus_stop_136_tr_alight = data_split(Pre_lock_route1_dir1_bus_stop_136_alight, 0.8,  'train')
Pre_lock_route1_dir1_bus_stop_136_test_alight = data_split(Pre_lock_route1_dir1_bus_stop_136_alight, 0.8,  'test')

Pre_lock_route1_dir1_bus_stop_1358_tr_alight = data_split(Pre_lock_route1_dir1_bus_stop_1358_alight, 0.8,  'train')
Pre_lock_route1_dir1_bus_stop_1358_test_alight = data_split(Pre_lock_route1_dir1_bus_stop_1358_alight, 0.8,  'test')

Pre_lock_route1_dir1_bus_stop_1487_tr_alight = data_split(Pre_lock_route1_dir1_bus_stop_1487_alight, 0.8,  'train')
Pre_lock_route1_dir1_bus_stop_1487_test_alight = data_split(Pre_lock_route1_dir1_bus_stop_1487_alight, 0.8,  'test')

Pre_lock_route1_dir1_bus_stop_1582_tr_alight = data_split(Pre_lock_route1_dir1_bus_stop_1582_alight, 0.8,  'train')
Pre_lock_route1_dir1_bus_stop_1582_test_alight = data_split(Pre_lock_route1_dir1_bus_stop_1582_alight, 0.8,  'test')

## Model Training and Selection

    * Training criteria: RMSE
    * Selection Criteria: AIC

In [100]:
Pre_lock_route1_dir1_bus_stop_124_model_selection = all_models_alight_cv(Pre_lock_route1_dir1_bus_stop_124_tr_alight,Pre_lock_route1_dir1_bus_stop_124_test_alight)
Pre_lock_route1_dir1_bus_stop_136_model_selection = all_models_alight_cv(Pre_lock_route1_dir1_bus_stop_136_tr_alight,Pre_lock_route1_dir1_bus_stop_136_test_alight)
Pre_lock_route1_dir1_bus_stop_1358_model_selection = all_models_alight_cv(Pre_lock_route1_dir1_bus_stop_1358_tr_alight,Pre_lock_route1_dir1_bus_stop_1358_test_alight)
Pre_lock_route1_dir1_bus_stop_1487_model_selection = all_models_alight_cv(Pre_lock_route1_dir1_bus_stop_1487_tr_alight,Pre_lock_route1_dir1_bus_stop_1487_test_alight)
Pre_lock_route1_dir1_bus_stop_1582_model_selection = all_models_alight_cv(Pre_lock_route1_dir1_bus_stop_1582_tr_alight,Pre_lock_route1_dir1_bus_stop_1582_test_alight)

ERROR: Error in unique(group): object 'Pre_lock_route1_dir1_bus_stop_124_tr_alight' not found


### Post-lockdown

In [142]:
Post_lock_route1_dir1_bus_stop_124_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Post_lock_rout1_dir1_bus_stop_124_alight.csv')
Post_lock_route1_dir1_bus_stop_124_alight = Post_lock_route1_dir1_bus_stop_124_alight[ , -c(1)]
Post_lock_route1_dir1_bus_stop_124_alight$board_count = round(Post_lock_route1_dir1_bus_stop_124_alight$alight_count)
Post_lock_route1_dir1_bus_stop_124_alight$month = factor(Post_lock_route1_dir1_bus_stop_124_alight$month)
Post_lock_route1_dir1_bus_stop_124_alight$service_kind = factor(Post_lock_route1_dir1_bus_stop_124_alight$service_kind)
Post_lock_route1_dir1_bus_stop_124_alight$hour = factor(Post_lock_route1_dir1_bus_stop_124_alight$hour)

Post_lock_route1_dir1_bus_stop_136_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Post_lock_rout1_dir1_bus_stop_136_alight.csv')
Post_lock_route1_dir1_bus_stop_136_alight = Post_lock_route1_dir1_bus_stop_136_alight[ , -c(1)]
Post_lock_route1_dir1_bus_stop_136_alight$board_count = round(Post_lock_route1_dir1_bus_stop_136_alight$alight_count)
Post_lock_route1_dir1_bus_stop_136_alight$month = factor(Post_lock_route1_dir1_bus_stop_136_alight$month)
Post_lock_route1_dir1_bus_stop_136_alight$service_kind = factor(Post_lock_route1_dir1_bus_stop_136_alight$service_kind)
Post_lock_route1_dir1_bus_stop_136_alight$hour = factor(Post_lock_route1_dir1_bus_stop_136_alight$hour)

Post_lock_route1_dir1_bus_stop_1358_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Post_lock_rout1_dir1_bus_stop_1358_alight.csv')
Post_lock_route1_dir1_bus_stop_1358_alight = Post_lock_route1_dir1_bus_stop_1358_alight[ , -c(1)]
Post_lock_route1_dir1_bus_stop_1358_alight$board_count = round(Post_lock_route1_dir1_bus_stop_1358_alight$alight_count)
Post_lock_route1_dir1_bus_stop_1358_alight$month = factor(Post_lock_route1_dir1_bus_stop_1358_alight$month)
Post_lock_route1_dir1_bus_stop_1358_alight$service_kind = factor(Post_lock_route1_dir1_bus_stop_1358_alight$service_kind)
Post_lock_route1_dir1_bus_stop_1358_alight$hour = factor(Post_lock_route1_dir1_bus_stop_1358_alight$hour)

Post_lock_route1_dir1_bus_stop_1487_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Post_lock_rout1_dir1_bus_stop_1487_alight.csv')
Post_lock_route1_dir1_bus_stop_1487_alight = Post_lock_route1_dir1_bus_stop_1487_alight[ , -c(1)]
Post_lock_route1_dir1_bus_stop_1487_alight$board_count = round(Post_lock_route1_dir1_bus_stop_1487_alight$alight_count)
Post_lock_route1_dir1_bus_stop_1487_alight$month = factor(Post_lock_route1_dir1_bus_stop_1487_alight$month)
Post_lock_route1_dir1_bus_stop_1487_alight$service_kind = factor(Post_lock_route1_dir1_bus_stop_1487_alight$service_kind)
Post_lock_route1_dir1_bus_stop_1487_alight$hour = factor(Post_lock_route1_dir1_bus_stop_1487_alight$hour)

Post_lock_route1_dir1_bus_stop_1582_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route1_dir1/Post_lock_rout1_dir1_bus_stop_1582_alight.csv')
Post_lock_route1_dir1_bus_stop_1582_alight = Post_lock_route1_dir1_bus_stop_1582_alight[ , -c(1)]
Post_lock_route1_dir1_bus_stop_1582_alight$board_count = round(Post_lock_route1_dir1_bus_stop_1582_alight$alight_count)
Post_lock_route1_dir1_bus_stop_1582_alight$month = factor(Post_lock_route1_dir1_bus_stop_1582_alight$month)
Post_lock_route1_dir1_bus_stop_1582_alight$service_kind = factor(Post_lock_route1_dir1_bus_stop_1582_alight$service_kind)
Post_lock_route1_dir1_bus_stop_1582_alight$hour = factor(Post_lock_route1_dir1_bus_stop_1582_alight$hour)

head(Post_lock_route1_dir1_bus_stop_136_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,weekday,0,0,0.1789358,0.02663446,0.00000000,0
3,weekday,0,0,0.1789358,0.02663446,0.03030303,0
3,weekday,0,0,0.1789358,0.02663446,0.06060606,0
3,weekday,0,0,0.1789358,0.02663446,0.15151515,0
3,weekday,0,0,0.1789358,0.02663446,0.18181818,0
3,weekday,0,0,0.1789358,0.02663446,0.27272727,0


### Data Partition into training and testing sets

* Training set: 80%
* Testing set: 20%

In [143]:
Post_lock_route1_dir1_bus_stop_124_tr_alight = data_split(Post_lock_route1_dir1_bus_stop_124_alight, 0.8,  'train')
Post_lock_route1_dir1_bus_stop_124_test_alight = data_split(Post_lock_route1_dir1_bus_stop_124_alight, 0.8,  'test')

Post_lock_route1_dir1_bus_stop_136_tr_alight = data_split(Post_lock_route1_dir1_bus_stop_136_alight, 0.8,  'train')
Post_lock_route1_dir1_bus_stop_136_test_alight = data_split(Post_lock_route1_dir1_bus_stop_136_alight, 0.8,  'test')

Post_lock_route1_dir1_bus_stop_1358_tr_alight = data_split(Post_lock_route1_dir1_bus_stop_1358_alight, 0.8,  'train')
Post_lock_route1_dir1_bus_stop_1358_test_alight = data_split(Post_lock_route1_dir1_bus_stop_1358_alight, 0.8,  'test')

Post_lock_route1_dir1_bus_stop_1487_tr_alight = data_split(Post_lock_route1_dir1_bus_stop_1487_alight, 0.8,  'train')
Post_lock_route1_dir1_bus_stop_1487_test_alight = data_split(Post_lock_route1_dir1_bus_stop_1487_alight, 0.8,  'test')

Post_lock_route1_dir1_bus_stop_1582_tr_alight = data_split(Post_lock_route1_dir1_bus_stop_1582_alight, 0.8,  'train')
Post_lock_route1_dir1_bus_stop_1582_test_alight = data_split(Post_lock_route1_dir1_bus_stop_1582_alight, 0.8,  'test')

### Model Training and Selection

    * Training criteria: RMSE
    * Selection Criteria: AIC

In [144]:
Post_lock_route1_dir1_bus_stop_124_model_selection = all_models_alight_cv(Post_lock_route1_dir1_bus_stop_124_tr_alight,Post_lock_route1_dir1_bus_stop_124_test_alight)
Post_lock_route1_dir1_bus_stop_136_model_selection = all_models_alight_cv(Post_lock_route1_dir1_bus_stop_136_tr_alight,Post_lock_route1_dir1_bus_stop_136_test_alight)
Post_lock_route1_dir1_bus_stop_1358_model_selection = all_models_alight_cv(Post_lock_route1_dir1_bus_stop_1358_tr_alight,Post_lock_route1_dir1_bus_stop_1358_test_alight)
Post_lock_route1_dir1_bus_stop_1487_model_selection = all_models_alight_cv(Post_lock_route1_dir1_bus_stop_1487_tr_alight,Post_lock_route1_dir1_bus_stop_1487_test_alight)
Post_lock_route1_dir1_bus_stop_1582_model_selection = all_models_alight_cv(Post_lock_route1_dir1_bus_stop_1582_tr_alight,Post_lock_route1_dir1_bus_stop_1582_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

In [145]:
2+2

[1] 4

## Route4 Direction 0

### Pre-lockdown

In [90]:
Pre_lock_route4_dir0_bus_stop_12_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_12_alight.csv')
Pre_lock_route4_dir0_bus_stop_12_alight = Pre_lock_route4_dir0_bus_stop_12_alight[ , -c(1)]
Pre_lock_route4_dir0_bus_stop_12_alight$board_count = round(Pre_lock_route4_dir0_bus_stop_12_alight$alight_count)
Pre_lock_route4_dir0_bus_stop_12_alight$month = factor(Pre_lock_route4_dir0_bus_stop_12_alight$month)
Pre_lock_route4_dir0_bus_stop_12_alight$service_kind = factor(Pre_lock_route4_dir0_bus_stop_12_alight$service_kind)
Pre_lock_route4_dir0_bus_stop_12_alight$hour = factor(Pre_lock_route4_dir0_bus_stop_12_alight$hour)

Pre_lock_route4_dir0_bus_stop_17_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_17_alight.csv')
Pre_lock_route4_dir0_bus_stop_17_alight = Pre_lock_route4_dir0_bus_stop_17_alight[ , -c(1)]
Pre_lock_route4_dir0_bus_stop_17_alight$board_count = round(Pre_lock_route4_dir0_bus_stop_17_alight$alight_count)
Pre_lock_route4_dir0_bus_stop_17_alight$month = factor(Pre_lock_route4_dir0_bus_stop_17_alight$month)
Pre_lock_route4_dir0_bus_stop_17_alight$service_kind = factor(Pre_lock_route4_dir0_bus_stop_17_alight$service_kind)
Pre_lock_route4_dir0_bus_stop_17_alight$hour = factor(Pre_lock_route4_dir0_bus_stop_17_alight$hour)

Pre_lock_route4_dir0_bus_stop_1351_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1351_alight.csv')
Pre_lock_route4_dir0_bus_stop_1351_alight = Pre_lock_route4_dir0_bus_stop_1351_alight[ , -c(1)]
Pre_lock_route4_dir0_bus_stop_1351_alight$board_count = round(Pre_lock_route4_dir0_bus_stop_1351_alight$alight_count)
Pre_lock_route4_dir0_bus_stop_1351_alight$month = factor(Pre_lock_route4_dir0_bus_stop_1351_alight$month)
Pre_lock_route4_dir0_bus_stop_1351_alight$service_kind = factor(Pre_lock_route4_dir0_bus_stop_1351_alight$service_kind)
Pre_lock_route4_dir0_bus_stop_1351_alight$hour = factor(Pre_lock_route4_dir0_bus_stop_1351_alight$hour)

Pre_lock_route4_dir0_bus_stop_1555_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1555_alight.csv')
Pre_lock_route4_dir0_bus_stop_1555_alight = Pre_lock_route4_dir0_bus_stop_1555_alight[ , -c(1)]
Pre_lock_route4_dir0_bus_stop_1555_alight$board_count = round(Pre_lock_route4_dir0_bus_stop_1555_alight$alight_count)
Pre_lock_route4_dir0_bus_stop_1555_alight$month = factor(Pre_lock_route4_dir0_bus_stop_1555_alight$month)
Pre_lock_route4_dir0_bus_stop_1555_alight$service_kind = factor(Pre_lock_route4_dir0_bus_stop_1555_alight$service_kind)
Pre_lock_route4_dir0_bus_stop_1555_alight$hour = factor(Pre_lock_route4_dir0_bus_stop_1555_alight$hour)

head(Pre_lock_route4_dir0_bus_stop_1555_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,4,0,0.179563,0.01575367,0.00000000,0
1,weekday,4,0,0.179563,0.01575367,0.02083333,0
1,weekday,4,0,0.179563,0.01575367,0.04166667,0
1,weekday,4,0,0.179563,0.01575367,0.06250000,0
1,weekday,4,0,0.179563,0.01575367,0.08333333,0
1,weekday,4,0,0.179563,0.01575367,0.12500000,0


### Data Partition into training and testing sets

* Training set: 80%
* Testing set: 20%

In [91]:
Pre_lock_route4_dir0_bus_stop_12_tr_alight = data_split(Pre_lock_route4_dir0_bus_stop_12_alight, 0.8,  'train')
Pre_lock_route4_dir0_bus_stop_12_test_alight = data_split(Pre_lock_route4_dir0_bus_stop_12_alight, 0.8,  'test')

Pre_lock_route4_dir0_bus_stop_17_tr_alight = data_split(Pre_lock_route4_dir0_bus_stop_17_alight, 0.8,  'train')
Pre_lock_route4_dir0_bus_stop_17_test_alight = data_split(Pre_lock_route4_dir0_bus_stop_17_alight, 0.8,  'test')

Pre_lock_route4_dir0_bus_stop_1351_tr_alight = data_split(Pre_lock_route4_dir0_bus_stop_1351_alight, 0.8,  'train')
Pre_lock_route4_dir0_bus_stop_1351_test_alight = data_split(Pre_lock_route4_dir0_bus_stop_1351_alight, 0.8,  'test')

Pre_lock_route4_dir0_bus_stop_1555_tr_alight = data_split(Pre_lock_route4_dir0_bus_stop_1555_alight, 0.8,  'train')
Pre_lock_route4_dir0_bus_stop_1555_test_alight = data_split(Pre_lock_route4_dir0_bus_stop_1555_alight, 0.8,  'test')

### Model Training and Selection

    * Training criteria: RMSE
    * Selection Criteria: AIC

In [92]:
Pre_lock_route4_dir0_bus_stop_12_model_selection = all_models_alight_cv(Pre_lock_route4_dir0_bus_stop_12_tr_alight,Pre_lock_route4_dir0_bus_stop_12_test_alight)
Pre_lock_route4_dir0_bus_stop_17_model_selection = all_models_alight_cv(Pre_lock_route4_dir0_bus_stop_17_tr_alight,Pre_lock_route4_dir0_bus_stop_17_test_alight)
Pre_lock_route4_dir0_bus_stop_1351_model_selection = all_models_alight_cv(Pre_lock_route4_dir0_bus_stop_1351_tr_alight,Pre_lock_route4_dir0_bus_stop_1351_test_alight)
Pre_lock_route4_dir0_bus_stop_1555_model_selection = all_models_alight_cv(Pre_lock_route4_dir0_bus_stop_1555_tr_alight,Pre_lock_route4_dir0_bus_stop_1555_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

## Post-lockdown

In [165]:
Post_lock_route4_dir0_bus_stop_12_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_12_alight.csv')
Post_lock_route4_dir0_bus_stop_12_alight = Post_lock_route4_dir0_bus_stop_12_alight[ , -c(1)]
Post_lock_route4_dir0_bus_stop_12_alight$board_count = round(Post_lock_route4_dir0_bus_stop_12_alight$alight_count)
Post_lock_route4_dir0_bus_stop_12_alight$month = factor(Post_lock_route4_dir0_bus_stop_12_alight$month)
Post_lock_route4_dir0_bus_stop_12_alight$service_kind = factor(Post_lock_route4_dir0_bus_stop_12_alight$service_kind)
Post_lock_route4_dir0_bus_stop_12_alight$hour = factor(Post_lock_route4_dir0_bus_stop_12_alight$hour)

Post_lock_route4_dir0_bus_stop_17_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_17_alight.csv')
Post_lock_route4_dir0_bus_stop_17_alight = Post_lock_route4_dir0_bus_stop_17_alight[ , -c(1)]
Post_lock_route4_dir0_bus_stop_17_alight$board_count = round(Post_lock_route4_dir0_bus_stop_17_alight$alight_count)
Post_lock_route4_dir0_bus_stop_17_alight$month = factor(Post_lock_route4_dir0_bus_stop_17_alight$month)
Post_lock_route4_dir0_bus_stop_17_alight$service_kind = factor(Post_lock_route4_dir0_bus_stop_17_alight$service_kind)
Post_lock_route4_dir0_bus_stop_17_alight$hour = factor(Post_lock_route4_dir0_bus_stop_17_alight$hour)

Post_lock_route4_dir0_bus_stop_1351_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1351_alight.csv')
Post_lock_route4_dir0_bus_stop_1351_alight = Post_lock_route4_dir0_bus_stop_1351_alight[ , -c(1)]
Post_lock_route4_dir0_bus_stop_1351_alight$board_count = round(Post_lock_route4_dir0_bus_stop_1351_alight$alight_count)
Post_lock_route4_dir0_bus_stop_1351_alight$month = factor(Post_lock_route4_dir0_bus_stop_1351_alight$month)
Post_lock_route4_dir0_bus_stop_1351_alight$service_kind = factor(Post_lock_route4_dir0_bus_stop_1351_alight$service_kind)
Post_lock_route4_dir0_bus_stop_1351_alight$hour = factor(Post_lock_route4_dir0_bus_stop_1351_alight$hour)

Post_lock_route4_dir0_bus_stop_1555_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1555_alight.csv')
Post_lock_route4_dir0_bus_stop_1555_alight = Post_lock_route4_dir0_bus_stop_1555_alight[ , -c(1)]
Post_lock_route4_dir0_bus_stop_1555_alight$board_count = round(Post_lock_route4_dir0_bus_stop_1555_alight$alight_count)
Post_lock_route4_dir0_bus_stop_1555_alight$month = factor(Post_lock_route4_dir0_bus_stop_1555_alight$month)
Post_lock_route4_dir0_bus_stop_1555_alight$service_kind = factor(Post_lock_route4_dir0_bus_stop_1555_alight$service_kind)
Post_lock_route4_dir0_bus_stop_1555_alight$hour = factor(Post_lock_route4_dir0_bus_stop_1555_alight$hour)

head(Post_lock_route4_dir0_bus_stop_1555_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,weekday,4,0,0.0736798,0.8538462,0.00000000,0
3,weekday,4,0,0.0736798,0.8538462,0.03030303,0
3,weekday,4,0,0.0736798,0.8538462,0.06060606,0
3,weekday,4,0,0.0736798,0.8538462,0.09090909,0
3,weekday,4,0,0.0736798,0.8538462,0.15151515,0
3,weekday,5,0,0.1917156,0.1455711,0.00000000,0


## Data Partition

In [167]:
Post_lock_route4_dir0_bus_stop_12_tr_alight = data_split(Post_lock_route4_dir0_bus_stop_12_alight, 0.8,  'train')
Post_lock_route4_dir0_bus_stop_12_test_alight = data_split(Post_lock_route4_dir0_bus_stop_12_alight, 0.8,  'test')

Post_lock_route4_dir0_bus_stop_17_tr_alight = data_split(Post_lock_route4_dir0_bus_stop_17_alight, 0.8,  'train')
Post_lock_route4_dir0_bus_stop_17_test_alight = data_split(Post_lock_route4_dir0_bus_stop_17_alight, 0.8,  'test')

Post_lock_route4_dir0_bus_stop_1351_tr_alight = data_split(Post_lock_route4_dir0_bus_stop_1351_alight, 0.8,  'train')
Post_lock_route4_dir0_bus_stop_1351_test_alight = data_split(Post_lock_route4_dir0_bus_stop_1351_alight, 0.8,  'test')

Post_lock_route4_dir0_bus_stop_1555_tr_alight = data_split(Post_lock_route4_dir0_bus_stop_1555_alight, 0.8,  'train')
Post_lock_route4_dir0_bus_stop_1555_test_alight = data_split(Post_lock_route4_dir0_bus_stop_1555_alight, 0.8,  'test')

## Model Training and Selection

In [ ]:
Post_lock_route4_dir0_bus_stop_12_model_selection = all_models_alight_cv(Post_lock_route4_dir0_bus_stop_12_tr_alight,Post_lock_route4_dir0_bus_stop_12_test_alight)
Post_lock_route4_dir0_bus_stop_17_model_selection = all_models_alight_cv(Post_lock_route4_dir0_bus_stop_17_tr_alight,Post_lock_route4_dir0_bus_stop_17_test_alight)
Post_lock_route4_dir0_bus_stop_1351_model_selection = all_models_alight_cv(Post_lock_route4_dir0_bus_stop_1351_tr_alight,Post_lock_route4_dir0_bus_stop_1351_test_alight)
Post_lock_route4_dir0_bus_stop_1555_model_selection = all_models_alight_cv(Post_lock_route4_dir0_bus_stop_1555_tr_alight,Post_lock_route4_dir0_bus_stop_1555_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

In [19]:
setwd('/home/jupyter/')

# Route 4 Direction 1

## Pre-lockdown

In [181]:
Pre_lock_route4_dir1_bus_stop_12_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_12_alight.csv')
Pre_lock_route4_dir1_bus_stop_12_alight = Pre_lock_route4_dir1_bus_stop_12_alight[ , -c(1)]
Pre_lock_route4_dir1_bus_stop_12_alight$board_count = round(Pre_lock_route4_dir1_bus_stop_12_alight$alight_count)
Pre_lock_route4_dir1_bus_stop_12_alight$month = factor(Pre_lock_route4_dir1_bus_stop_12_alight$month)
Pre_lock_route4_dir1_bus_stop_12_alight$service_kind = factor(Pre_lock_route4_dir1_bus_stop_12_alight$service_kind)
Pre_lock_route4_dir1_bus_stop_12_alight$hour = factor(Pre_lock_route4_dir1_bus_stop_12_alight$hour)

Pre_lock_route4_dir1_bus_stop_17_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_17_alight.csv')
Pre_lock_route4_dir1_bus_stop_17_alight = Pre_lock_route4_dir1_bus_stop_17_alight[ , -c(1)]
Pre_lock_route4_dir1_bus_stop_17_alight$board_count = round(Pre_lock_route4_dir1_bus_stop_17_alight$alight_count)
Pre_lock_route4_dir1_bus_stop_17_alight$month = factor(Pre_lock_route4_dir1_bus_stop_17_alight$month)
Pre_lock_route4_dir1_bus_stop_17_alight$service_kind = factor(Pre_lock_route4_dir1_bus_stop_17_alight$service_kind)
Pre_lock_route4_dir1_bus_stop_17_alight$hour = factor(Pre_lock_route4_dir1_bus_stop_17_alight$hour)

Pre_lock_route4_dir1_bus_stop_1351_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1351_alight.csv')
Pre_lock_route4_dir1_bus_stop_1351_alight = Pre_lock_route4_dir1_bus_stop_1351_alight[ , -c(1)]
Pre_lock_route4_dir1_bus_stop_1351_alight$board_count = round(Pre_lock_route4_dir1_bus_stop_1351_alight$alight_count)
Pre_lock_route4_dir1_bus_stop_1351_alight$month = factor(Pre_lock_route4_dir1_bus_stop_1351_alight$month)
Pre_lock_route4_dir1_bus_stop_1351_alight$service_kind = factor(Pre_lock_route4_dir1_bus_stop_1351_alight$service_kind)
Pre_lock_route4_dir1_bus_stop_1351_alight$hour = factor(Pre_lock_route4_dir1_bus_stop_1351_alight$hour)

Pre_lock_route4_dir1_bus_stop_1555_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1555_alight.csv')
Pre_lock_route4_dir1_bus_stop_1555_alight = Pre_lock_route4_dir1_bus_stop_1555_alight[ , -c(1)]
Pre_lock_route4_dir1_bus_stop_1555_alight$board_count = round(Pre_lock_route4_dir1_bus_stop_1555_alight$alight_count)
Pre_lock_route4_dir1_bus_stop_1555_alight$month = factor(Pre_lock_route4_dir1_bus_stop_1555_alight$month)
Pre_lock_route4_dir1_bus_stop_1555_alight$service_kind = factor(Pre_lock_route4_dir1_bus_stop_1555_alight$service_kind)
Pre_lock_route4_dir1_bus_stop_1555_alight$hour = factor(Pre_lock_route4_dir1_bus_stop_1555_alight$hour)

head(Pre_lock_route4_dir1_bus_stop_1555_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,0,0,0,0,0.00000000,0
1,weekday,0,0,0,0,0.02083333,0
1,weekday,0,0,0,0,0.04166667,0
1,weekday,0,0,0,0,0.06250000,0
1,weekday,0,0,0,0,0.08333333,0
1,weekday,0,0,0,0,0.10416667,0


## Data Partition

In [188]:
Pre_lock_route4_dir1_bus_stop_12_tr_alight = data_split(Pre_lock_route4_dir1_bus_stop_12_alight, 0.8,  'train')
Pre_lock_route4_dir1_bus_stop_12_test_alight = data_split(Pre_lock_route4_dir1_bus_stop_12_alight, 0.8,  'test')

Pre_lock_route4_dir1_bus_stop_17_tr_alight = data_split(Pre_lock_route4_dir1_bus_stop_17_alight, 0.8,  'train')
Pre_lock_route4_dir1_bus_stop_17_test_alight = data_split(Pre_lock_route4_dir1_bus_stop_17_alight, 0.8,  'test')

Pre_lock_route4_dir1_bus_stop_1351_tr_alight = data_split(Pre_lock_route4_dir1_bus_stop_1351_alight, 0.8,  'train')
Pre_lock_route4_dir1_bus_stop_1351_test_alight = data_split(Pre_lock_route4_dir1_bus_stop_1351_alight, 0.8,  'test')

Pre_lock_route4_dir1_bus_stop_1555_tr_alight = data_split(Pre_lock_route4_dir1_bus_stop_1555_alight, 0.8,  'train')
Pre_lock_route4_dir1_bus_stop_1555_test_alight = data_split(Pre_lock_route4_dir1_bus_stop_1555_alight, 0.8,  'test')

In [ ]:
all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_17_tr_alight, Pre_lock_route4_dir1_bus_stop_17_test_alight)

Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.500000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.300000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.300000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.200000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.200000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.100000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.200000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.600000”
Warning message in dpois(y, mu, log = TRUE):
“non-integer x = 0.

## Model Training and Selection

In [196]:
#Pre_lock_route4_dir1_bus_stop_12_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_12_tr_alight,Pre_lock_route4_dir1_bus_stop_12_test_alight)
#Pre_lock_route4_dir1_bus_stop_17_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_17_tr_alight, Pre_lock_route4_dir1_bus_stop_17_test_alight)
#Pre_lock_route4_dir1_bus_stop_1351_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_1351_tr_alight,Pre_lock_route4_dir1_bus_stop_1351_test_alight)
#Pre_lock_route4_dir1_bus_stop_1555_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_1555_tr_alight,Pre_lock_route4_dir1_bus_stop_1555_test_alight)

In [199]:
str(Pre_lock_route4_dir1_bus_stop_17_model_selection)

ERROR: Error in str(Pre_lock_route4_dir1_bus_stop_17_model_selection): object 'Pre_lock_route4_dir1_bus_stop_17_model_selection' not found


## Post-lockdown

In [148]:
Post_lock_route4_dir1_bus_stop_12_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_12_alight.csv')
Post_lock_route4_dir1_bus_stop_12_alight = Post_lock_route4_dir1_bus_stop_12_alight[ , -c(1)]
Post_lock_route4_dir1_bus_stop_12_alight$board_count = round(Post_lock_route4_dir1_bus_stop_12_alight$alight_count)
Post_lock_route4_dir1_bus_stop_12_alight$month = factor(Post_lock_route4_dir1_bus_stop_12_alight$month)
Post_lock_route4_dir1_bus_stop_12_alight$service_kind = factor(Post_lock_route4_dir1_bus_stop_12_alight$service_kind)
Post_lock_route4_dir1_bus_stop_12_alight$hour = factor(Post_lock_route4_dir1_bus_stop_12_alight$hour)

Post_lock_route4_dir1_bus_stop_17_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_17_alight.csv')
Post_lock_route4_dir1_bus_stop_17_alight = Post_lock_route4_dir1_bus_stop_17_alight[ , -c(1)]
Post_lock_route4_dir1_bus_stop_17_alight$board_count = round(Post_lock_route4_dir1_bus_stop_17_alight$alight_count)
Post_lock_route4_dir1_bus_stop_17_alight$month = factor(Post_lock_route4_dir1_bus_stop_17_alight$month)
Post_lock_route4_dir1_bus_stop_17_alight$service_kind = factor(Post_lock_route4_dir1_bus_stop_17_alight$service_kind)
Post_lock_route4_dir1_bus_stop_17_alight$hour = factor(Post_lock_route4_dir1_bus_stop_17_alight$hour)

Post_lock_route4_dir1_bus_stop_1351_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1351_alight.csv')
Post_lock_route4_dir1_bus_stop_1351_alight = Post_lock_route4_dir1_bus_stop_1351_alight[ , -c(1)]
Post_lock_route4_dir1_bus_stop_1351_alight$board_count = round(Post_lock_route4_dir1_bus_stop_1351_alight$alight_count)
Post_lock_route4_dir1_bus_stop_1351_alight$month = factor(Post_lock_route4_dir1_bus_stop_1351_alight$month)
Post_lock_route4_dir1_bus_stop_1351_alight$service_kind = factor(Post_lock_route4_dir1_bus_stop_1351_alight$service_kind)
Post_lock_route4_dir1_bus_stop_1351_alight$hour = factor(Post_lock_route4_dir1_bus_stop_1351_alight$hour)

Post_lock_route4_dir1_bus_stop_1555_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1555_alight.csv')
Post_lock_route4_dir1_bus_stop_1555_alight = Post_lock_route4_dir1_bus_stop_1555_alight[ , -c(1)]
Post_lock_route4_dir1_bus_stop_1555_alight$board_count = round(Post_lock_route4_dir1_bus_stop_1555_alight$alight_count)
Post_lock_route4_dir1_bus_stop_1555_alight$month = factor(Post_lock_route4_dir1_bus_stop_1555_alight$month)
Post_lock_route4_dir1_bus_stop_1555_alight$service_kind = factor(Post_lock_route4_dir1_bus_stop_1555_alight$service_kind)
Post_lock_route4_dir1_bus_stop_1555_alight$hour = factor(Post_lock_route4_dir1_bus_stop_1555_alight$hour)

head(Post_lock_route4_dir1_bus_stop_1555_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,weekday,0,0,0.4030824,0,0.00000000,0
3,weekday,0,0,0.4030824,0,0.03030303,0
3,weekday,0,0,0.4030824,0,0.06060606,0
3,weekday,0,0,0.4030824,0,0.15151515,0
3,weekday,0,0,0.4030824,0,0.18181818,0
3,weekday,0,0,0.4030824,0,0.27272727,0


### Data Partition into training and testing sets

* Training set: 80%
* Testing set: 20%

In [149]:
Post_lock_route4_dir1_bus_stop_12_tr_alight = data_split(Post_lock_route4_dir1_bus_stop_12_alight, 0.8,  'train')
Post_lock_route4_dir1_bus_stop_12_test_alight = data_split(Post_lock_route4_dir1_bus_stop_12_alight, 0.8,  'test')

Post_lock_route4_dir1_bus_stop_17_tr_alight = data_split(Post_lock_route4_dir1_bus_stop_17_alight, 0.8,  'train')
Post_lock_route4_dir1_bus_stop_17_test_alight = data_split(Post_lock_route4_dir1_bus_stop_17_alight, 0.8,  'test')

Post_lock_route4_dir1_bus_stop_1351_tr_alight = data_split(Post_lock_route4_dir1_bus_stop_1351_alight, 0.8,  'train')
Post_lock_route4_dir1_bus_stop_1351_test_alight = data_split(Post_lock_route4_dir1_bus_stop_1351_alight, 0.8,  'test')

Post_lock_route4_dir1_bus_stop_1555_tr_alight = data_split(Post_lock_route4_dir1_bus_stop_1555_alight, 0.8,  'train')
Post_lock_route4_dir1_bus_stop_1555_test_alight = data_split(Post_lock_route4_dir1_bus_stop_1555_alight, 0.8,  'test')

## Model Training and Selection

    * Training criteria: RMSE
    * Selection Criteria: AIC

In [ ]:
Post_lock_route4_dir1_bus_stop_12_model_selection = all_models_alight_cv(Post_lock_route4_dir1_bus_stop_12_tr_alight,Post_lock_route4_dir1_bus_stop_12_test_alight)
Post_lock_route4_dir1_bus_stop_17_model_selection = all_models_alight_cv(Post_lock_route4_dir1_bus_stop_17_tr_alight,Post_lock_route4_dir1_bus_stop_17_test_alight)
Post_lock_route4_dir1_bus_stop_1351_model_selection = all_models_alight_cv(Post_lock_route4_dir1_bus_stop_1351_tr_alight,Post_lock_route4_dir1_bus_stop_1351_test_alight)
Post_lock_route4_dir1_bus_stop_1555_model_selection = all_models_alight_cv(Post_lock_route4_dir1_bus_stop_1555_tr_alight,Post_lock_route4_dir1_bus_stop_1555_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

# Route 9 Direction 0

## Pre-lockdown

In [171]:
Pre_lock_route9_dir0_bus_stop_1710_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir0/Pre_lock_route9_dir0_bus_stop_1710_alight.csv')
Pre_lock_route9_dir0_bus_stop_1710_alight = Pre_lock_route9_dir0_bus_stop_1710_alight[ , -c(1)]
Pre_lock_route9_dir0_bus_stop_1710_alight$board_count = round(Pre_lock_route9_dir0_bus_stop_1710_alight$alight_count)
Pre_lock_route9_dir0_bus_stop_1710_alight$month = factor(Pre_lock_route9_dir0_bus_stop_1710_alight$month)
Pre_lock_route9_dir0_bus_stop_1710_alight$service_kind = factor(Pre_lock_route9_dir0_bus_stop_1710_alight$service_kind)
Pre_lock_route9_dir0_bus_stop_1710_alight$hour = factor(Pre_lock_route9_dir0_bus_stop_1710_alight$hour)

head(Pre_lock_route9_dir0_bus_stop_1710_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)




month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,5,0,0.1146047,0.1719148,0.00000000,0
1,weekday,5,0,0.1146047,0.1719148,0.02083333,0
1,weekday,5,0,0.1146047,0.1719148,0.04166667,0
1,weekday,5,0,0.1146047,0.1719148,0.06250000,0
1,weekday,5,0,0.1146047,0.1719148,0.08333333,0
1,weekday,5,0,0.1146047,0.1719148,0.10416667,0


## Data Partition

In [172]:
Pre_lock_route9_dir0_bus_stop_1710_tr_alight = data_split(Pre_lock_route9_dir0_bus_stop_1710_alight, 0.8,  'train')
Pre_lock_route9_dir0_bus_stop_1710_test_alight = data_split(Pre_lock_route9_dir0_bus_stop_1710_alight, 0.8,  'test')

## Model Selection

In [173]:
Pre_lock_route9_dir0_bus_stop_1710_model_selection = all_models_alight_cv(Pre_lock_route9_dir0_bus_stop_1710_tr_alight,Pre_lock_route9_dir0_bus_stop_1710_test_alight)
#Pre_lock_route4_dir1_bus_stop_17_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_17_tr_alight,Pre_lock_route4_dir1_bus_stop_17_test_alight)
#Pre_lock_route4_dir1_bus_stop_1351_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_1351_tr_alight,Pre_lock_route4_dir1_bus_stop_1351_test_alight)
#Pre_lock_route4_dir1_bus_stop_1555_model_selection = all_models_alight_cv(Pre_lock_route4_dir1_bus_stop_1555_tr_alight,Pre_lock_route4_dir1_bus_stop_1555_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

## Post-lockdown

In [175]:
Post_lock_route9_dir0_bus_stop_1710_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir0/Post_lock_route9_dir0_bus_stop_1710_alight.csv')
Post_lock_route9_dir0_bus_stop_1710_alight = Post_lock_route9_dir0_bus_stop_1710_alight[ , -c(1)]
Post_lock_route9_dir0_bus_stop_1710_alight$board_count = round(Post_lock_route9_dir0_bus_stop_1710_alight$alight_count)
Post_lock_route9_dir0_bus_stop_1710_alight$month = factor(Post_lock_route9_dir0_bus_stop_1710_alight$month)
Post_lock_route9_dir0_bus_stop_1710_alight$service_kind = factor(Post_lock_route9_dir0_bus_stop_1710_alight$service_kind)
Post_lock_route9_dir0_bus_stop_1710_alight$hour = factor(Post_lock_route9_dir0_bus_stop_1710_alight$hour)

head(Post_lock_route9_dir0_bus_stop_1710_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)




month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,weekday,5,0,0.3941382,0.07786241,0.00000000,0
3,weekday,5,0,0.3941382,0.07786241,0.03030303,0
3,weekday,5,0,0.3941382,0.07786241,0.06060606,0
3,weekday,5,0,0.3941382,0.07786241,0.09090909,0
3,weekday,5,0,0.3941382,0.07786241,0.12121212,0
3,weekday,5,0,0.3941382,0.07786241,0.15151515,0


In [7]:
setwd('/home/jupyter/')

In [8]:
getwd()

[1] "/home/jupyter"

## Route 9 Direction 1

### Pre-lockdown

In [10]:
Pre_lock_route9_dir1_bus_stop_1177_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight.csv')
Pre_lock_route9_dir1_bus_stop_1177_alight = Pre_lock_route9_dir1_bus_stop_1177_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1177_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1177_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1177_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$month)
Pre_lock_route9_dir1_bus_stop_1177_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1177_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$hour)

Pre_lock_route9_dir1_bus_stop_1178_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight.csv')
Pre_lock_route9_dir1_bus_stop_1178_alight = Pre_lock_route9_dir1_bus_stop_1178_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1178_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1178_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1178_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$month)
Pre_lock_route9_dir1_bus_stop_1178_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1178_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$hour)

Pre_lock_route9_dir1_bus_stop_1181_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight.csv')
Pre_lock_route9_dir1_bus_stop_1181_alight = Pre_lock_route9_dir1_bus_stop_1181_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1181_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1181_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1181_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$month)
Pre_lock_route9_dir1_bus_stop_1181_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1181_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$hour)

Pre_lock_route9_dir1_bus_stop_1504_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight.csv')
Pre_lock_route9_dir1_bus_stop_1504_alight = Pre_lock_route9_dir1_bus_stop_1504_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1504_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1504_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1504_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$month)
Pre_lock_route9_dir1_bus_stop_1504_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1504_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$hour)

Pre_lock_route9_dir1_bus_stop_1710_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight.csv')
Pre_lock_route9_dir1_bus_stop_1710_alight = Pre_lock_route9_dir1_bus_stop_1710_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1710_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1710_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1710_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$month)
Pre_lock_route9_dir1_bus_stop_1710_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1710_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$hour)

head(Pre_lock_route9_dir1_bus_stop_1710_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,5,0,0.1366726,0.1086803,0.00000000,0
1,weekday,5,0,0.1366726,0.1086803,0.02083333,0
1,weekday,5,0,0.1366726,0.1086803,0.04166667,0
1,weekday,5,0,0.1366726,0.1086803,0.06250000,0
1,weekday,5,0,0.1366726,0.1086803,0.08333333,0
1,weekday,5,0,0.1366726,0.1086803,0.10416667,0


## Data partition 

In [93]:
Pre_lock_route9_dir1_bus_stop_1177_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1177_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1178_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1178_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1181_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1181_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1504_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1504_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1710_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1710_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'test')

ERROR: Error in data_split(Pre_lock_route9_dir1_bus_stop_1177_alight, 0.8, "train"): object 'Pre_lock_route9_dir1_bus_stop_1177_alight' not found


## Model training and selection

In [14]:
Pre_lock_route4_dir1_bus_stop_1177_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1177_tr_alight,Pre_lock_route9_dir1_bus_stop_1177_test_alight)
Pre_lock_route4_dir1_bus_stop_1178_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1178_tr_alight,Pre_lock_route9_dir1_bus_stop_1178_test_alight)
Pre_lock_route4_dir1_bus_stop_1181_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1181_tr_alight,Pre_lock_route9_dir1_bus_stop_1181_test_alight)
Pre_lock_route4_dir1_bus_stop_1504_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1504_tr_alight,Pre_lock_route9_dir1_bus_stop_1504_test_alight)
Pre_lock_route4_dir1_bus_stop_1710_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1710_tr_alight,Pre_lock_route9_dir1_bus_stop_1710_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

# Route 9 Direction 1

## Pre-lockdown

In [123]:
Pre_lock_route9_dir1_bus_stop_1177_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight.csv')
Pre_lock_route9_dir1_bus_stop_1177_alight = Pre_lock_route9_dir1_bus_stop_1177_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1177_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1177_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1177_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$month)
Pre_lock_route9_dir1_bus_stop_1177_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1177_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1177_alight$hour)

Pre_lock_route9_dir1_bus_stop_1178_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight.csv')
Pre_lock_route9_dir1_bus_stop_1178_alight = Pre_lock_route9_dir1_bus_stop_1178_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1178_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1178_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1178_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$month)
Pre_lock_route9_dir1_bus_stop_1178_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1178_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1178_alight$hour)

Pre_lock_route9_dir1_bus_stop_1181_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight.csv')
Pre_lock_route9_dir1_bus_stop_1181_alight = Pre_lock_route9_dir1_bus_stop_1181_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1181_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1181_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1181_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$month)
Pre_lock_route9_dir1_bus_stop_1181_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1181_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1181_alight$hour)

Pre_lock_route9_dir1_bus_stop_1504_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight.csv')
Pre_lock_route9_dir1_bus_stop_1504_alight = Pre_lock_route9_dir1_bus_stop_1504_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1504_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1504_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1504_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$month)
Pre_lock_route9_dir1_bus_stop_1504_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1504_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1504_alight$hour)

Pre_lock_route9_dir1_bus_stop_1710_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight.csv')
Pre_lock_route9_dir1_bus_stop_1710_alight = Pre_lock_route9_dir1_bus_stop_1710_alight[ , -c(1)]
Pre_lock_route9_dir1_bus_stop_1710_alight$board_count = round(Pre_lock_route9_dir1_bus_stop_1710_alight$alight_count)
Pre_lock_route9_dir1_bus_stop_1710_alight$month = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$month)
Pre_lock_route9_dir1_bus_stop_1710_alight$service_kind = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$service_kind)
Pre_lock_route9_dir1_bus_stop_1710_alight$hour = factor(Pre_lock_route9_dir1_bus_stop_1710_alight$hour)

head(Pre_lock_route9_dir1_bus_stop_1504_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,weekday,5,0,0.2328434,0.1133349,0.00000000,0
1,weekday,5,0,0.2328434,0.1133349,0.02083333,0
1,weekday,5,0,0.2328434,0.1133349,0.04166667,0
1,weekday,5,0,0.2328434,0.1133349,0.06250000,0
1,weekday,5,0,0.2328434,0.1133349,0.08333333,0
1,weekday,5,0,0.2328434,0.1133349,0.10416667,0


## Data Partition

In [124]:
Pre_lock_route9_dir1_bus_stop_1177_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1177_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1178_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1178_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1181_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1181_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1504_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1504_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'test')

Pre_lock_route9_dir1_bus_stop_1710_tr_alight = data_split(Pre_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'train')
Pre_lock_route9_dir1_bus_stop_1710_test_alight = data_split(Pre_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'test')

## Model Training and Selection

In [125]:
Pre_lock_route9_dir1_bus_stop_1177_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1177_tr_alight, Pre_lock_route9_dir1_bus_stop_1177_test_alight)
Pre_lock_route9_dir1_bus_stop_1178_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1178_tr_alight, Pre_lock_route9_dir1_bus_stop_1178_test_alight)
Pre_lock_route9_dir1_bus_stop_1181_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1181_tr_alight, Pre_lock_route9_dir1_bus_stop_1181_test_alight)
Pre_lock_route9_dir1_bus_stop_1504_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1504_tr_alight, Pre_lock_route9_dir1_bus_stop_1504_test_alight)
Pre_lock_route9_dir1_bus_stop_1710_model_selection = all_models_alight_cv(Pre_lock_route9_dir1_bus_stop_1710_tr_alight, Pre_lock_route9_dir1_bus_stop_1710_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

### Post-lockdown

In [30]:
Post_lock_route9_dir1_bus_stop_1177_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1177_alight.csv')
Post_lock_route9_dir1_bus_stop_1177_alight = Post_lock_route9_dir1_bus_stop_1177_alight[ , -c(1)]
Post_lock_route9_dir1_bus_stop_1177_alight$board_count = round(Post_lock_route9_dir1_bus_stop_1177_alight$alight_count)
Post_lock_route9_dir1_bus_stop_1177_alight$month = factor(Post_lock_route9_dir1_bus_stop_1177_alight$month)
Post_lock_route9_dir1_bus_stop_1177_alight$service_kind = factor(Post_lock_route9_dir1_bus_stop_1177_alight$service_kind)
Post_lock_route9_dir1_bus_stop_1177_alight$hour = factor(Post_lock_route9_dir1_bus_stop_1177_alight$hour)

Post_lock_route9_dir1_bus_stop_1178_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1178_alight.csv')
Post_lock_route9_dir1_bus_stop_1178_alight = Post_lock_route9_dir1_bus_stop_1178_alight[ , -c(1)]
Post_lock_route9_dir1_bus_stop_1178_alight$board_count = round(Post_lock_route9_dir1_bus_stop_1178_alight$alight_count)
Post_lock_route9_dir1_bus_stop_1178_alight$month = factor(Post_lock_route9_dir1_bus_stop_1178_alight$month)
Post_lock_route9_dir1_bus_stop_1178_alight$service_kind = factor(Post_lock_route9_dir1_bus_stop_1178_alight$service_kind)
Post_lock_route9_dir1_bus_stop_1178_alight$hour = factor(Post_lock_route9_dir1_bus_stop_1178_alight$hour)

Post_lock_route9_dir1_bus_stop_1181_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1181_alight.csv')
Post_lock_route9_dir1_bus_stop_1181_alight = Post_lock_route9_dir1_bus_stop_1181_alight[ , -c(1)]
Post_lock_route9_dir1_bus_stop_1181_alight$board_count = round(Post_lock_route9_dir1_bus_stop_1181_alight$alight_count)
Post_lock_route9_dir1_bus_stop_1181_alight$month = factor(Post_lock_route9_dir1_bus_stop_1181_alight$month)
Post_lock_route9_dir1_bus_stop_1181_alight$service_kind = factor(Post_lock_route9_dir1_bus_stop_1181_alight$service_kind)
Post_lock_route9_dir1_bus_stop_1181_alight$hour = factor(Post_lock_route9_dir1_bus_stop_1181_alight$hour)

Post_lock_route9_dir1_bus_stop_1504_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1504_alight.csv')
Post_lock_route9_dir1_bus_stop_1504_alight = Post_lock_route9_dir1_bus_stop_1504_alight[ , -c(1)]
Post_lock_route9_dir1_bus_stop_1504_alight$board_count = round(Post_lock_route9_dir1_bus_stop_1504_alight$alight_count)
Post_lock_route9_dir1_bus_stop_1504_alight$month = factor(Post_lock_route9_dir1_bus_stop_1504_alight$month)
Post_lock_route9_dir1_bus_stop_1504_alight$service_kind = factor(Post_lock_route9_dir1_bus_stop_1504_alight$service_kind)
Post_lock_route9_dir1_bus_stop_1504_alight$hour = factor(Post_lock_route9_dir1_bus_stop_1504_alight$hour)

Post_lock_route9_dir1_bus_stop_1710_alight = read_csv('/home/jupyter/Data_for_regression/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1710_alight.csv')
Post_lock_route9_dir1_bus_stop_1710_alight = Post_lock_route9_dir1_bus_stop_1710_alight[ , -c(1)]
Post_lock_route9_dir1_bus_stop_1710_alight$board_count = round(Post_lock_route9_dir1_bus_stop_1710_alight$alight_count)
Post_lock_route9_dir1_bus_stop_1710_alight$month = factor(Post_lock_route9_dir1_bus_stop_1710_alight$month)
Post_lock_route9_dir1_bus_stop_1710_alight$service_kind = factor(Post_lock_route9_dir1_bus_stop_1710_alight$service_kind)
Post_lock_route9_dir1_bus_stop_1710_alight$hour = factor(Post_lock_route9_dir1_bus_stop_1710_alight$hour)

head(Post_lock_route9_dir1_bus_stop_1504_alight)

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  alight_count = col_double(),
  temp = col_double(),
  precip = col_double(),
  surrounding_alight_count = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),


month,service_kind,hour,alight_count,temp,precip,surrounding_alight_count,board_count
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,weekday,5,0,0.3057928,0.1111111,0.00000000,0
3,weekday,5,0,0.3057928,0.1111111,0.03030303,0
3,weekday,5,0,0.3057928,0.1111111,0.06060606,0
3,weekday,5,0,0.3057928,0.1111111,0.09090909,0
3,weekday,5,0,0.3057928,0.1111111,0.12121212,0
3,weekday,5,0,0.3057928,0.1111111,0.15151515,0


In [31]:
Post_lock_route9_dir1_bus_stop_1177_tr_alight = data_split(Post_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'train')
Post_lock_route9_dir1_bus_stop_1177_test_alight = data_split(Post_lock_route9_dir1_bus_stop_1177_alight, 0.8,  'test')

Post_lock_route9_dir1_bus_stop_1178_tr_alight = data_split(Post_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'train')
Post_lock_route9_dir1_bus_stop_1178_test_alight = data_split(Post_lock_route9_dir1_bus_stop_1178_alight, 0.8,  'test')

Post_lock_route9_dir1_bus_stop_1181_tr_alight = data_split(Post_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'train')
Post_lock_route9_dir1_bus_stop_1181_test_alight = data_split(Post_lock_route9_dir1_bus_stop_1181_alight, 0.8,  'test')

Post_lock_route9_dir1_bus_stop_1504_tr_alight = data_split(Post_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'train')
Post_lock_route9_dir1_bus_stop_1504_test_alight = data_split(Post_lock_route9_dir1_bus_stop_1504_alight, 0.8,  'test')

Post_lock_route9_dir1_bus_stop_1710_tr_alight = data_split(Post_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'train')
Post_lock_route9_dir1_bus_stop_1710_test_alight = data_split(Post_lock_route9_dir1_bus_stop_1710_alight, 0.8,  'test')

In [32]:
Post_lock_route9_dir1_bus_stop_1177_model_selection = all_models_alight_cv(Post_lock_route9_dir1_bus_stop_1177_tr_alight, Post_lock_route9_dir1_bus_stop_1177_test_alight)
Post_lock_route9_dir1_bus_stop_1178_model_selection = all_models_alight_cv(Post_lock_route9_dir1_bus_stop_1178_tr_alight, Post_lock_route9_dir1_bus_stop_1178_test_alight)
Post_lock_route9_dir1_bus_stop_1181_model_selection = all_models_alight_cv(Post_lock_route9_dir1_bus_stop_1181_tr_alight, Post_lock_route9_dir1_bus_stop_1181_test_alight)
Post_lock_route9_dir1_bus_stop_1504_model_selection = all_models_alight_cv(Post_lock_route9_dir1_bus_stop_1504_tr_alight, Post_lock_route9_dir1_bus_stop_1504_test_alight)
Post_lock_route9_dir1_bus_stop_1710_model_selection = all_models_alight_cv(Post_lock_route9_dir1_bus_stop_1710_tr_alight, Post_lock_route9_dir1_bus_stop_1710_test_alight)

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in theta.ml(Y, m

In [96]:
Post_lock_route9_dir1_bus_stop_1177_model_selection[['train_RMSEs']]

Poiss,QPoiss,NB,ZIP_Poiss,ZIP_NB,hurdle
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.2121437,0.2121437,0.2121487,0.2112455,0.2112458,0.325132


In [97]:
Post_lock_route9_dir1_bus_stop_1177_model_selection[['test_RMSEs']]

Poiss,QPoiss,NB,ZIP_Poiss,ZIP_NB,hurdle
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.3942623,0.3942623,0.3943284,0.3792397,0.3792336,0.4166761


In [ ]:
        performance <- list("train_RMSEs" = Train_RMSE,
                            "test_RMSEs" = Test_RMSE,
                            "Models_AIC" = Models_AIC,
                            "Selected_Model" = Selected_Model,
                            "Trained_Models" = Models,
                            "Predicted" = Predictions)

In [101]:
Post_lock_route9_dir1_bus_stop_1177_model_selection[['Selected_Model']]

character(0)

In [8]:
get_Performance = function(dtfrm, metric){
    
    if(metric == 'RMSE'){
        Train_RMSEs = dtfrm[['train_RMSEs']]
        Test_RMSEs = dtfrm[['test_RMSEs']]
    }
    else{
        
    }
    
    return()
}

# Results

In [33]:
get_Chart = function(dtfrm, dtfrm_test){
    Best_Model = dtfrm[['Selected_Model']]
    Chart = dtfrm_test[ , c(1, 2, 3, 4)]
    if(Best_Model == 'Poisson'){
        Chart$Predicted = dtfrm[['Predicted']][ , c(1)]
        return(Chart)
    }
    else if(Best_Model == 'Negative_Binomial'){
        Chart$Predicted = dtfrm[['Predicted']][ , c(2)]
        return(Chart)
    }
    else if(Best_Model == 'ZIP_Poisson'){
        Chart$Predicted = dtfrm[['Predicted']][ , c(3)]
        return(Chart)
    }
    else if(Best_Model == 'ZIP_Negative_Binomial'){
        Chart$Predicted = dtfrm[['Predicted']][ , c(4)]
        return(Chart)
    }
    else if(Best_Model == 'Hurdle_Poisson_and_Binomial'){
        Chart$Predicted = dtfrm[['Predicted']][ , c(5)]
        return(Chart)
    }
}


In [34]:
get_Performance = function(dtfrm){
    Best_Model = dtfrm[['Selected_Model']]
    
    if(Best_Model == 'Poisson'){
        train_rmse = dtfrm[['train_RMSEs']][ , c(1)]
        test_rmse = dtfrm[['test_RMSEs']][ , c(1)]
        Performance = data.frame(cbind('Train_RMSE' = train_rmse, 'Test_RMSE' = test_rmse))
        rownames(Performance) = c('Poisson')
        return(Performance)
    }
    else if(Best_Model == 'Negative_Binomial'){
        train_rmse = dtfrm[['train_RMSEs']][ , c(2)]
        test_rmse = dtfrm[['test_RMSEs']][ , c(2)]
        Performance = data.frame(cbind('Train_RMSE' = train_rmse, 'Test_RMSE' = test_rmse))
        rownames(Performance) = c('Negative_Binomial')
        return(Performance)
    }
    else if(Best_Model == 'ZIP_Poisson'){
        train_rmse = dtfrm[['train_RMSEs']][ , c(3)]
        test_rmse = dtfrm[['test_RMSEs']][ , c(3)]
        Performance = data.frame(cbind('Train_RMSE' = train_rmse, 'Test_RMSE' = test_rmse))
        rownames(Performance) = c('ZIP_Poisson')
        return(Performance)
    }
    else if(Best_Model == 'ZIP_Negative_Binomial'){
        train_rmse = dtfrm[['train_RMSEs']][ , c(4)]
        test_rmse = dtfrm[['test_RMSEs']][ , c(4)]
        Performance = data.frame(cbind('Train_RMSE' = train_rmse, 'Test_RMSE' = test_rmse))
        rownames(Performance) = c('ZIP_Negative_Binomial')
        return(Performance)
    }
    else if(Best_Model == 'Hurdle_Poisson_and_Binomial'){
        train_rmse = dtfrm[['train_RMSEs']][ , c(5)]
        test_rmse = dtfrm[['test_RMSEs']][ , c(5)]
        Performance = data.frame(cbind('Train_RMSE' = train_rmse, 'Test_RMSE' = test_rmse))
        rownames(Performance) = c('Hurdle_Poisson_and_Binomial')
        return(Performance)
    }    
}

In [35]:
get_Model_Selection = function(dtfrm){
    AIC_scores = dtfrm[['Models_AIC']]
    colnames(AIC_scores) = c('Model_Name', 'AIC')
    AIC_scores$Model_Name = as.character(AIC_scores$Model_Name)
    AIC_scores$AIC = round(as.numeric(as.character(AIC_scores$AIC)),3)
    return(AIC_scores)
}

## Route 1 Direction 1

### Pre-lockdown

In [136]:
Pre_lock_route1_dir1_bus_stop_124_alight_Model_Selection = get_Model_Selection(Pre_lock_route1_dir1_bus_stop_124_model_selection)
Pre_lock_route1_dir1_bus_stop_124_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route1_dir1_bus_stop_124_model_selection)
Pre_lock_route1_dir1_bus_stop_124_alight_Performance = get_Performance(Pre_lock_route1_dir1_bus_stop_124_model_selection)
Pre_lock_route1_dir1_bus_stop_124_alight_Chart = get_Chart(Pre_lock_route1_dir1_bus_stop_124_model_selection, Pre_lock_route1_dir1_bus_stop_124_test_alight)

write.csv(Pre_lock_route1_dir1_bus_stop_124_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_124_alight_Model_Selection.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_124_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_124_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_124_alight_Performance, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_124_alight_Performance.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_124_alight_Chart, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_124_alight_Chart.csv')

In [137]:
Pre_lock_route1_dir1_bus_stop_136_alight_Model_Selection = get_Model_Selection(Pre_lock_route1_dir1_bus_stop_136_model_selection)
Pre_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route1_dir1_bus_stop_136_model_selection)
Pre_lock_route1_dir1_bus_stop_136_alight_Performance = get_Performance(Pre_lock_route1_dir1_bus_stop_136_model_selection)
Pre_lock_route1_dir1_bus_stop_136_alight_Chart = get_Chart(Pre_lock_route1_dir1_bus_stop_136_model_selection, Pre_lock_route1_dir1_bus_stop_136_test_alight)

write.csv(Pre_lock_route1_dir1_bus_stop_136_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_136_alight_Model_Selection.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_136_alight_Performance, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_136_alight_Performance.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_136_alight_Chart, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_136_alight_Chart.csv')

In [138]:
Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Selection = get_Model_Selection(Pre_lock_route1_dir1_bus_stop_1358_model_selection)
Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route1_dir1_bus_stop_1358_model_selection)
Pre_lock_route1_dir1_bus_stop_1358_alight_Performance = get_Performance(Pre_lock_route1_dir1_bus_stop_1358_model_selection)
Pre_lock_route1_dir1_bus_stop_1358_alight_Chart = get_Chart(Pre_lock_route1_dir1_bus_stop_1358_model_selection, Pre_lock_route1_dir1_bus_stop_1358_test_alight)

write.csv(Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Selection.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1358_alight_Performance, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1358_alight_Performance.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1358_alight_Chart, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1358_alight_Chart.csv')

In [139]:
Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Selection = get_Model_Selection(Pre_lock_route1_dir1_bus_stop_1487_model_selection)
Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route1_dir1_bus_stop_1487_model_selection)
Pre_lock_route1_dir1_bus_stop_1487_alight_Performance = get_Performance(Pre_lock_route1_dir1_bus_stop_1487_model_selection)
Pre_lock_route1_dir1_bus_stop_1487_alight_Chart = get_Chart(Pre_lock_route1_dir1_bus_stop_1487_model_selection, Pre_lock_route1_dir1_bus_stop_1487_test_alight)

write.csv(Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Selection.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1487_alight_Performance, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1487_alight_Performance.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1487_alight_Chart, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1487_alight_Chart.csv')

In [140]:
Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Selection = get_Model_Selection(Pre_lock_route1_dir1_bus_stop_1582_model_selection)
Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route1_dir1_bus_stop_1582_model_selection)
Pre_lock_route1_dir1_bus_stop_1582_alight_Performance = get_Performance(Pre_lock_route1_dir1_bus_stop_1582_model_selection)
Pre_lock_route1_dir1_bus_stop_1582_alight_Chart = get_Chart(Pre_lock_route1_dir1_bus_stop_1582_model_selection, Pre_lock_route1_dir1_bus_stop_1582_test_alight)

write.csv(Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Selection.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1582_alight_Performance, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1582_alight_Performance.csv')
write.csv(Pre_lock_route1_dir1_bus_stop_1582_alight_Chart, 'Results/Alight_Count/route1_dir1/Pre_lock_route1_dir1_bus_stop_1582_alight_Chart.csv')

### Pre-lockdown

In [152]:
Post_lock_route1_dir1_bus_stop_124_alight_Model_Selection = get_Model_Selection(Post_lock_route1_dir1_bus_stop_124_model_selection)
Post_lock_route1_dir1_bus_stop_124_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route1_dir1_bus_stop_124_model_selection)
Post_lock_route1_dir1_bus_stop_124_alight_Performance = get_Performance(Post_lock_route1_dir1_bus_stop_124_model_selection)
Post_lock_route1_dir1_bus_stop_124_alight_Chart = get_Chart(Post_lock_route1_dir1_bus_stop_124_model_selection, Post_lock_route1_dir1_bus_stop_124_test_alight)

write.csv(Post_lock_route1_dir1_bus_stop_124_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_124_alight_Model_Selection.csv')
write.csv(Post_lock_route1_dir1_bus_stop_124_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_124_alight_Model_Coefficients.csv')
write.csv(Post_lock_route1_dir1_bus_stop_124_alight_Performance, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_124_alight_Performance.csv')
write.csv(Post_lock_route1_dir1_bus_stop_124_alight_Chart, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_124_alight_Chart.csv')

In [154]:
Post_lock_route1_dir1_bus_stop_136_alight_Model_Selection = get_Model_Selection(Post_lock_route1_dir1_bus_stop_136_model_selection)
Post_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients = get_Model_Selection(Post_lock_route1_dir1_bus_stop_136_model_selection)
Post_lock_route1_dir1_bus_stop_136_alight_Performance = get_Performance(Post_lock_route1_dir1_bus_stop_136_model_selection)
Post_lock_route1_dir1_bus_stop_136_alight_Chart = get_Chart(Post_lock_route1_dir1_bus_stop_136_model_selection, Post_lock_route1_dir1_bus_stop_136_test_alight)

write.csv(Post_lock_route1_dir1_bus_stop_136_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_136_alight_Model_Selection.csv')
write.csv(Post_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_136_alight_Model_Coefficients.csv')
write.csv(Post_lock_route1_dir1_bus_stop_136_alight_Performance, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_136_alight_Performance.csv')
write.csv(Post_lock_route1_dir1_bus_stop_136_alight_Chart, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_136_alight_Chart.csv')

In [153]:
Post_lock_route1_dir1_bus_stop_1358_alight_Model_Selection = get_Model_Selection(Post_lock_route1_dir1_bus_stop_1358_model_selection)
Post_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route1_dir1_bus_stop_1358_model_selection)
Post_lock_route1_dir1_bus_stop_1358_alight_Performance = get_Performance(Post_lock_route1_dir1_bus_stop_1358_model_selection)
Post_lock_route1_dir1_bus_stop_1358_alight_Chart = get_Chart(Post_lock_route1_dir1_bus_stop_1358_model_selection, Post_lock_route1_dir1_bus_stop_1358_test_alight)

write.csv(Post_lock_route1_dir1_bus_stop_1358_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1358_alight_Model_Selection.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1358_alight_Model_Coefficients.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1358_alight_Performance, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1358_alight_Performance.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1358_alight_Chart, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1358_alight_Chart.csv')

In [155]:
Post_lock_route1_dir1_bus_stop_1487_alight_Model_Selection = get_Model_Selection(Post_lock_route1_dir1_bus_stop_1487_model_selection)
Post_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route1_dir1_bus_stop_1487_model_selection)
Post_lock_route1_dir1_bus_stop_1487_alight_Performance = get_Performance(Post_lock_route1_dir1_bus_stop_1487_model_selection)
Post_lock_route1_dir1_bus_stop_1487_alight_Chart = get_Chart(Post_lock_route1_dir1_bus_stop_1487_model_selection, Post_lock_route1_dir1_bus_stop_1487_test_alight)

write.csv(Post_lock_route1_dir1_bus_stop_1487_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1487_alight_Model_Selection.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1487_alight_Model_Coefficients.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1487_alight_Performance, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1487_alight_Performance.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1487_alight_Chart, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1487_alight_Chart.csv')

In [157]:
Post_lock_route1_dir1_bus_stop_1582_alight_Model_Selection = get_Model_Selection(Post_lock_route1_dir1_bus_stop_1582_model_selection)
Post_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route1_dir1_bus_stop_1582_model_selection)
Post_lock_route1_dir1_bus_stop_1582_alight_Performance = get_Performance(Post_lock_route1_dir1_bus_stop_1582_model_selection)
Post_lock_route1_dir1_bus_stop_1582_alight_Chart = get_Chart(Post_lock_route1_dir1_bus_stop_1582_model_selection, Post_lock_route1_dir1_bus_stop_1582_test_alight)

write.csv(Post_lock_route1_dir1_bus_stop_1582_alight_Model_Selection, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1582_alight_Model_Selection.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1582_alight_Model_Coefficients.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1582_alight_Performance, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1582_alight_Performance.csv')
write.csv(Post_lock_route1_dir1_bus_stop_1582_alight_Chart, 'Results/Alight_Count/route1_dir1/Post_lock_route1_dir1_bus_stop_1582_alight_Chart.csv')

## Route 4 Direciton 0

### Pre-lockdown

In [159]:
Pre_lock_route4_dir0_bus_stop_12_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir0_bus_stop_12_model_selection)
Pre_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir0_bus_stop_12_model_selection)
Pre_lock_route4_dir0_bus_stop_12_alight_Performance = get_Performance(Pre_lock_route4_dir0_bus_stop_12_model_selection)
Pre_lock_route4_dir0_bus_stop_12_alight_Chart = get_Chart(Pre_lock_route4_dir0_bus_stop_12_model_selection, Pre_lock_route4_dir0_bus_stop_12_test_alight)

write.csv(Pre_lock_route4_dir0_bus_stop_12_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_12_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_12_alight_Performance, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_12_alight_Performance.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_12_alight_Chart, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_12_alight_Chart.csv')

In [161]:
Pre_lock_route4_dir0_bus_stop_17_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir0_bus_stop_17_model_selection)
Pre_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir0_bus_stop_17_model_selection)
Pre_lock_route4_dir0_bus_stop_17_alight_Performance = get_Performance(Pre_lock_route4_dir0_bus_stop_17_model_selection)
Pre_lock_route4_dir0_bus_stop_17_alight_Chart = get_Chart(Pre_lock_route4_dir0_bus_stop_17_model_selection, Pre_lock_route4_dir0_bus_stop_17_test_alight)

write.csv(Pre_lock_route4_dir0_bus_stop_17_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_17_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_17_alight_Performance, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_17_alight_Performance.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_17_alight_Chart, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_17_alight_Chart.csv')

In [162]:
Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir0_bus_stop_1351_model_selection)
Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir0_bus_stop_1351_model_selection)
Pre_lock_route4_dir0_bus_stop_1351_alight_Performance = get_Performance(Pre_lock_route4_dir0_bus_stop_1351_model_selection)
Pre_lock_route4_dir0_bus_stop_1351_alight_Chart = get_Chart(Pre_lock_route4_dir0_bus_stop_1351_model_selection, Pre_lock_route4_dir0_bus_stop_1351_test_alight)

write.csv(Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1351_alight_Performance, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1351_alight_Performance.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1351_alight_Chart, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1351_alight_Chart.csv')

In [163]:
Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir0_bus_stop_1555_model_selection)
Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir0_bus_stop_1555_model_selection)
Pre_lock_route4_dir0_bus_stop_1555_alight_Performance = get_Performance(Pre_lock_route4_dir0_bus_stop_1555_model_selection)
Pre_lock_route4_dir0_bus_stop_1555_alight_Chart = get_Chart(Pre_lock_route4_dir0_bus_stop_1555_model_selection, Pre_lock_route4_dir0_bus_stop_1555_test_alight)

write.csv(Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1555_alight_Performance, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1555_alight_Performance.csv')
write.csv(Pre_lock_route4_dir0_bus_stop_1555_alight_Chart, 'Results/Alight_Count/route4_dir0/Pre_lock_route4_dir0_bus_stop_1555_alight_Chart.csv')

### Post-lockdown

In [169]:
Post_lock_route4_dir0_bus_stop_12_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir0_bus_stop_12_model_selection)
Post_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir0_bus_stop_12_model_selection)
Post_lock_route4_dir0_bus_stop_12_alight_Performance = get_Performance(Post_lock_route4_dir0_bus_stop_12_model_selection)
Post_lock_route4_dir0_bus_stop_12_alight_Chart = get_Chart(Post_lock_route4_dir0_bus_stop_12_model_selection, Post_lock_route4_dir0_bus_stop_12_test_alight)

write.csv(Post_lock_route4_dir0_bus_stop_12_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_12_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_12_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir0_bus_stop_12_alight_Performance, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_12_alight_Performance.csv')
write.csv(Post_lock_route4_dir0_bus_stop_12_alight_Chart, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_12_alight_Chart.csv')

In [170]:
Post_lock_route4_dir0_bus_stop_17_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir0_bus_stop_17_model_selection)
Post_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir0_bus_stop_17_model_selection)
Post_lock_route4_dir0_bus_stop_17_alight_Performance = get_Performance(Post_lock_route4_dir0_bus_stop_17_model_selection)
Post_lock_route4_dir0_bus_stop_17_alight_Chart = get_Chart(Post_lock_route4_dir0_bus_stop_17_model_selection, Post_lock_route4_dir0_bus_stop_17_test_alight)

write.csv(Post_lock_route4_dir0_bus_stop_17_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_17_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_17_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir0_bus_stop_17_alight_Performance, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_17_alight_Performance.csv')
write.csv(Post_lock_route4_dir0_bus_stop_17_alight_Chart, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_17_alight_Chart.csv')

In [171]:
Post_lock_route4_dir0_bus_stop_1351_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir0_bus_stop_1351_model_selection)
Post_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir0_bus_stop_1351_model_selection)
Post_lock_route4_dir0_bus_stop_1351_alight_Performance = get_Performance(Post_lock_route4_dir0_bus_stop_1351_model_selection)
Post_lock_route4_dir0_bus_stop_1351_alight_Chart = get_Chart(Post_lock_route4_dir0_bus_stop_1351_model_selection, Post_lock_route4_dir0_bus_stop_1351_test_alight)

write.csv(Post_lock_route4_dir0_bus_stop_1351_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1351_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1351_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1351_alight_Performance, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1351_alight_Performance.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1351_alight_Chart, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1351_alight_Chart.csv')

In [172]:
Post_lock_route4_dir0_bus_stop_1555_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir0_bus_stop_1555_model_selection)
Post_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir0_bus_stop_1555_model_selection)
Post_lock_route4_dir0_bus_stop_1555_alight_Performance = get_Performance(Post_lock_route4_dir0_bus_stop_1555_model_selection)
Post_lock_route4_dir0_bus_stop_1555_alight_Chart = get_Chart(Post_lock_route4_dir0_bus_stop_1555_model_selection, Post_lock_route4_dir0_bus_stop_1555_test_alight)

write.csv(Post_lock_route4_dir0_bus_stop_1555_alight_Model_Selection, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1555_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1555_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1555_alight_Performance, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1555_alight_Performance.csv')
write.csv(Post_lock_route4_dir0_bus_stop_1555_alight_Chart, 'Results/Alight_Count/route4_dir0/Post_lock_route4_dir0_bus_stop_1555_alight_Chart.csv')

# Route 4 Direction 1

## Pre-lockdown

In [173]:
Pre_lock_route4_dir1_bus_stop_12_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir1_bus_stop_12_model_selection)
Pre_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir1_bus_stop_12_model_selection)
Pre_lock_route4_dir1_bus_stop_12_alight_Performance = get_Performance(Pre_lock_route4_dir1_bus_stop_12_model_selection)
Pre_lock_route4_dir1_bus_stop_12_alight_Chart = get_Chart(Pre_lock_route4_dir1_bus_stop_12_model_selection, Pre_lock_route4_dir1_bus_stop_12_test_alight)

write.csv(Pre_lock_route4_dir1_bus_stop_12_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_12_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_12_alight_Performance, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_12_alight_Performance.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_12_alight_Chart, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_12_alight_Chart.csv')

In [183]:
Pre_lock_route4_dir1_bus_stop_17_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir1_bus_stop_17_model_selection)
Pre_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir1_bus_stop_17_model_selection)
Pre_lock_route4_dir1_bus_stop_17_alight_Performance = get_Performance(Pre_lock_route4_dir1_bus_stop_17_model_selection)
Pre_lock_route4_dir1_bus_stop_17_alight_Chart = get_Chart(Pre_lock_route4_dir1_bus_stop_17_model_selection, Pre_lock_route4_dir1_bus_stop_17_test_alight)

write.csv(Pre_lock_route4_dir1_bus_stop_17_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_17_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_17_alight_Performance, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_17_alight_Performance.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_17_alight_Chart, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_17_alight_Chart.csv')

ERROR: Error in get_Model_Selection(Pre_lock_route4_dir1_bus_stop_17_model_selection): object 'Pre_lock_route4_dir1_bus_stop_17_model_selection' not found


In [184]:
str(Pre_lock_route4_dir1_bus_stop_17_model_selection)

ERROR: Error in str(Pre_lock_route4_dir1_bus_stop_17_model_selection): object 'Pre_lock_route4_dir1_bus_stop_17_model_selection' not found


In [185]:
Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir1_bus_stop_1351_model_selection)
Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir1_bus_stop_1351_model_selection)
Pre_lock_route4_dir1_bus_stop_1351_alight_Performance = get_Performance(Pre_lock_route4_dir1_bus_stop_1351_model_selection)
Pre_lock_route4_dir1_bus_stop_1351_alight_Chart = get_Chart(Pre_lock_route4_dir1_bus_stop_1351_model_selection, Pre_lock_route4_dir1_bus_stop_1351_test_alight)

write.csv(Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1351_alight_Performance, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1351_alight_Performance.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1351_alight_Chart, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1351_alight_Chart.csv')

In [186]:
Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Selection = get_Model_Selection(Pre_lock_route4_dir1_bus_stop_1555_model_selection)
Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route4_dir1_bus_stop_1555_model_selection)
Pre_lock_route4_dir1_bus_stop_1555_alight_Performance = get_Performance(Pre_lock_route4_dir1_bus_stop_1555_model_selection)
Pre_lock_route4_dir1_bus_stop_1555_alight_Chart = get_Chart(Pre_lock_route4_dir1_bus_stop_1555_model_selection, Pre_lock_route4_dir1_bus_stop_1555_test_alight)

write.csv(Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Selection.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1555_alight_Performance, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1555_alight_Performance.csv')
write.csv(Pre_lock_route4_dir1_bus_stop_1555_alight_Chart, 'Results/Alight_Count/route4_dir1/Pre_lock_route4_dir1_bus_stop_1555_alight_Chart.csv')

# Route 4 Direction 1

## Post-lockdown

In [147]:
get_Performance(Post_lock_route4_dir1_bus_stop_12_model_selection)

ERROR: Error in get_Performance(Post_lock_route4_dir1_bus_stop_12_model_selection): object 'Post_lock_route4_dir1_bus_stop_12_model_selection' not found


In [175]:
Post_lock_route4_dir1_bus_stop_12_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir1_bus_stop_12_model_selection)
Post_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir1_bus_stop_12_model_selection)
Post_lock_route4_dir1_bus_stop_12_alight_Performance = get_Performance(Post_lock_route4_dir1_bus_stop_12_model_selection)
Post_lock_route4_dir1_bus_stop_12_alight_Chart = get_Chart(Post_lock_route4_dir1_bus_stop_12_model_selection, Post_lock_route4_dir1_bus_stop_12_test_alight)

write.csv(Post_lock_route4_dir1_bus_stop_12_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_12_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_12_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir1_bus_stop_12_alight_Performance, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_12_alight_Performance.csv')
write.csv(Post_lock_route4_dir1_bus_stop_12_alight_Chart, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_12_alight_Chart.csv')

In [177]:
Post_lock_route4_dir1_bus_stop_17_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir1_bus_stop_17_model_selection)
Post_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir1_bus_stop_17_model_selection)
Post_lock_route4_dir1_bus_stop_17_alight_Performance = get_Performance(Post_lock_route4_dir1_bus_stop_17_model_selection)
Post_lock_route4_dir1_bus_stop_17_alight_Chart = get_Chart(Post_lock_route4_dir1_bus_stop_17_model_selection, Post_lock_route4_dir1_bus_stop_17_test_alight)

write.csv(Post_lock_route4_dir1_bus_stop_17_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_17_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_17_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir1_bus_stop_17_alight_Performance, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_17_alight_Performance.csv')
write.csv(Post_lock_route4_dir1_bus_stop_17_alight_Chart, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_17_alight_Chart.csv')

In [178]:
Post_lock_route4_dir1_bus_stop_1351_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir1_bus_stop_1351_model_selection)
Post_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir1_bus_stop_1351_model_selection)
Post_lock_route4_dir1_bus_stop_1351_alight_Performance = get_Performance(Post_lock_route4_dir1_bus_stop_1351_model_selection)
Post_lock_route4_dir1_bus_stop_1351_alight_Chart = get_Chart(Post_lock_route4_dir1_bus_stop_1351_model_selection, Post_lock_route4_dir1_bus_stop_1351_test_alight)

write.csv(Post_lock_route4_dir1_bus_stop_1351_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1351_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1351_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1351_alight_Performance, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1351_alight_Performance.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1351_alight_Chart, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1351_alight_Chart.csv')

In [179]:
Post_lock_route4_dir1_bus_stop_1555_alight_Model_Selection = get_Model_Selection(Post_lock_route4_dir1_bus_stop_1555_model_selection)
Post_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route4_dir1_bus_stop_1555_model_selection)
Post_lock_route4_dir1_bus_stop_1555_alight_Performance = get_Performance(Post_lock_route4_dir1_bus_stop_1555_model_selection)
Post_lock_route4_dir1_bus_stop_1555_alight_Chart = get_Chart(Post_lock_route4_dir1_bus_stop_1555_model_selection, Post_lock_route4_dir1_bus_stop_1555_test_alight)

write.csv(Post_lock_route4_dir1_bus_stop_1555_alight_Model_Selection, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1555_alight_Model_Selection.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1555_alight_Model_Coefficients.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1555_alight_Performance, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1555_alight_Performance.csv')
write.csv(Post_lock_route4_dir1_bus_stop_1555_alight_Chart, 'Results/Alight_Count/route4_dir1/Post_lock_route4_dir1_bus_stop_1555_alight_Chart.csv')

# Route 9 dir 0

## Pre-lockdown

In [180]:
Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir0_bus_stop_1177_model_selection)
Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir0_bus_stop_1177_model_selection)
Pre_lock_route9_dir0_bus_stop_1177_alight_Performance = get_Performance(Pre_lock_route9_dir0_bus_stop_1177_model_selection)
Pre_lock_route9_dir0_bus_stop_1177_alight_Chart = get_Chart(Pre_lock_route9_dir0_bus_stop_1177_model_selection, Pre_lock_route9_dir0_bus_stop_1177_test_alight)

write.csv(Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Selection, 'Results/Alight_Count/route9_dir0/Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir0/Pre_lock_route9_dir0_bus_stop_1177_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir0_bus_stop_1177_alight_Performance, 'Results/Alight_Count/route9_dir0/Pre_lock_route9_dir0_bus_stop_1177_alight_Performance.csv')
write.csv(Pre_lock_route9_dir0_bus_stop_1177_alight_Chart, 'Results/Alight_Count/route9_dir0/Pre_lock_route9_dir0_bus_stop_1177_alight_Chart.csv')

ERROR: Error in get_Model_Selection(Pre_lock_route9_dir0_bus_stop_1177_model_selection): object 'Pre_lock_route9_dir0_bus_stop_1177_model_selection' not found


In [127]:
2+2

[1] 4

# Route 9 dir 1

## Pre-lockdown

In [133]:
Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir1_bus_stop_1177_model_selection)
Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir1_bus_stop_1177_model_selection)
Pre_lock_route9_dir1_bus_stop_1177_alight_Performance = get_Performance(Pre_lock_route9_dir1_bus_stop_1177_model_selection)
Pre_lock_route9_dir1_bus_stop_1177_alight_Chart = get_Chart(Pre_lock_route9_dir1_bus_stop_1177_model_selection, Pre_lock_route9_dir1_bus_stop_1177_test_alight)

write.csv(Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1177_alight_Performance, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight_Performance.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1177_alight_Chart, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1177_alight_Chart.csv')

In [130]:
Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir1_bus_stop_1178_model_selection)
Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir1_bus_stop_1178_model_selection)
Pre_lock_route9_dir1_bus_stop_1178_alight_Performance = get_Performance(Pre_lock_route9_dir1_bus_stop_1178_model_selection)
Pre_lock_route9_dir1_bus_stop_1178_alight_Chart = get_Chart(Pre_lock_route9_dir1_bus_stop_1178_model_selection, Pre_lock_route9_dir1_bus_stop_1178_test_alight)

write.csv(Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1178_alight_Performance, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight_Performance.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1178_alight_Chart, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1178_alight_Chart.csv')

In [131]:
Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir1_bus_stop_1181_model_selection)
Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir1_bus_stop_1181_model_selection)
Pre_lock_route9_dir1_bus_stop_1181_alight_Performance = get_Performance(Pre_lock_route9_dir1_bus_stop_1181_model_selection)
Pre_lock_route9_dir1_bus_stop_1181_alight_Chart = get_Chart(Pre_lock_route9_dir1_bus_stop_1181_model_selection, Pre_lock_route9_dir1_bus_stop_1181_test_alight)

write.csv(Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1181_alight_Performance, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight_Performance.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1181_alight_Chart, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1181_alight_Chart.csv')

In [132]:
Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir1_bus_stop_1504_model_selection)
Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir1_bus_stop_1504_model_selection)
Pre_lock_route9_dir1_bus_stop_1504_alight_Performance = get_Performance(Pre_lock_route9_dir1_bus_stop_1504_model_selection)
Pre_lock_route9_dir1_bus_stop_1504_alight_Chart = get_Chart(Pre_lock_route9_dir1_bus_stop_1504_model_selection, Pre_lock_route9_dir1_bus_stop_1504_test_alight)

write.csv(Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1504_alight_Performance, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight_Performance.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1504_alight_Chart, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1504_alight_Chart.csv')

In [134]:
Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Selection = get_Model_Selection(Pre_lock_route9_dir1_bus_stop_1710_model_selection)
Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients = get_Model_Coeff(Pre_lock_route9_dir1_bus_stop_1710_model_selection)
Pre_lock_route9_dir1_bus_stop_1710_alight_Performance = get_Performance(Pre_lock_route9_dir1_bus_stop_1710_model_selection)
Pre_lock_route9_dir1_bus_stop_1710_alight_Chart = get_Chart(Pre_lock_route9_dir1_bus_stop_1710_model_selection, Pre_lock_route9_dir1_bus_stop_1710_test_alight)

write.csv(Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Selection.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1710_alight_Performance, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight_Performance.csv')
write.csv(Pre_lock_route9_dir1_bus_stop_1710_alight_Chart, 'Results/Alight_Count/route9_dir1/Pre_lock_route9_dir1_bus_stop_1710_alight_Chart.csv')

### Post-lockdown

In [135]:
2+2

[1] 4

In [117]:
Post_lock_route9_dir1_bus_stop_1177_alight_Model_Selection = get_Model_Selection(Post_lock_route9_dir1_bus_stop_1177_model_selection)
Post_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route9_dir1_bus_stop_1177_model_selection)
Post_lock_route9_dir1_bus_stop_1177_alight_Performance = get_Performance(Post_lock_route9_dir1_bus_stop_1177_model_selection)
Post_lock_route9_dir1_bus_stop_1177_alight_Chart = get_Chart(Post_lock_route9_dir1_bus_stop_1177_model_selection, Post_lock_route9_dir1_bus_stop_1177_test_alight)

write.csv(Post_lock_route9_dir1_bus_stop_1177_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1177_alight_Model_Selection.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1177_alight_Model_Coefficients.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1177_alight_Performance, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1177_alight_Performance.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1177_alight_Chart, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1177_alight_Chart.csv')

In [118]:
Post_lock_route9_dir1_bus_stop_1178_alight_Model_Selection = get_Model_Selection(Post_lock_route9_dir1_bus_stop_1178_model_selection)
Post_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route9_dir1_bus_stop_1178_model_selection)
Post_lock_route9_dir1_bus_stop_1178_alight_Performance = get_Performance(Post_lock_route9_dir1_bus_stop_1178_model_selection)
Post_lock_route9_dir1_bus_stop_1178_alight_Chart = get_Chart(Post_lock_route9_dir1_bus_stop_1178_model_selection, Post_lock_route9_dir1_bus_stop_1178_test_alight)

write.csv(Post_lock_route9_dir1_bus_stop_1178_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1178_alight_Model_Selection.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1178_alight_Model_Coefficients.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1178_alight_Performance, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1178_alight_Performance.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1178_alight_Chart, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1178_alight_Chart.csv')

In [119]:
Post_lock_route9_dir1_bus_stop_1181_alight_Model_Selection = get_Model_Selection(Post_lock_route9_dir1_bus_stop_1181_model_selection)
Post_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route9_dir1_bus_stop_1181_model_selection)
Post_lock_route9_dir1_bus_stop_1181_alight_Performance = get_Performance(Post_lock_route9_dir1_bus_stop_1181_model_selection)
Post_lock_route9_dir1_bus_stop_1181_alight_Chart = get_Chart(Post_lock_route9_dir1_bus_stop_1181_model_selection, Post_lock_route9_dir1_bus_stop_1181_test_alight)

write.csv(Post_lock_route9_dir1_bus_stop_1181_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1181_alight_Model_Selection.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1181_alight_Model_Coefficients.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1181_alight_Performance, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1181_alight_Performance.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1181_alight_Chart, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1181_alight_Chart.csv')

In [120]:
Post_lock_route9_dir1_bus_stop_1504_alight_Model_Selection = get_Model_Selection(Post_lock_route9_dir1_bus_stop_1504_model_selection)
Post_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route9_dir1_bus_stop_1504_model_selection)
Post_lock_route9_dir1_bus_stop_1504_alight_Performance = get_Performance(Post_lock_route9_dir1_bus_stop_1504_model_selection)
Post_lock_route9_dir1_bus_stop_1504_alight_Chart = get_Chart(Post_lock_route9_dir1_bus_stop_1504_model_selection, Post_lock_route9_dir1_bus_stop_1504_test_alight)

write.csv(Post_lock_route9_dir1_bus_stop_1504_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1504_alight_Model_Selection.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1504_alight_Model_Coefficients.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1504_alight_Performance, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1504_alight_Performance.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1504_alight_Chart, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1504_alight_Chart.csv')

In [121]:
Post_lock_route9_dir1_bus_stop_1710_alight_Model_Selection = get_Model_Selection(Post_lock_route9_dir1_bus_stop_1710_model_selection)
Post_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients = get_Model_Coeff(Post_lock_route9_dir1_bus_stop_1710_model_selection)
Post_lock_route9_dir1_bus_stop_1710_alight_Performance = get_Performance(Post_lock_route9_dir1_bus_stop_1710_model_selection)
Post_lock_route9_dir1_bus_stop_1710_alight_Chart = get_Chart(Post_lock_route9_dir1_bus_stop_1710_model_selection, Post_lock_route9_dir1_bus_stop_1710_test_alight)

write.csv(Post_lock_route9_dir1_bus_stop_1710_alight_Model_Selection, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1710_alight_Model_Selection.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1710_alight_Model_Coefficients.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1710_alight_Performance, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1710_alight_Performance.csv')
write.csv(Post_lock_route9_dir1_bus_stop_1710_alight_Chart, 'Results/Alight_Count/route9_dir1/Post_lock_route9_dir1_bus_stop_1710_alight_Chart.csv')